In [1]:
## run-1
## download data sku dari tatanama
## common problems: slow server pyany & make sure path to file

import time
import pandas as pd
import numpy as np
import math

import smtplib

from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
from email.mime.multipart import MIMEMultipart
from smtplib import SMTP
import smtplib
import sys
import requests
import os


print('Reading Data SKU')
data_SKU = pd.read_excel('SKU_File\data_SKU.xlsx')
print('Scrapping Tatanama')
s = requests.Session()
s.get("https://tatanama.pythonanywhere.com")
s.post("https://tatanama.pythonanywhere.com", data = {'username' : 'ecommerce', 'password' : 'ecommerce'})
r = s.get("https://tatanama.pythonanywhere.com/download")

with open('SKU_File\Master tatanama.xlsx', 'wb') as output:
    output.write(r.content)

print('ok')

if os.path.isfile('SKU_File\Master tatanama.xlsx') :    
    SKU_append = pd.read_excel('SKU_File\Master tatanama.xlsx', engine='openpyxl')
    SKU_append.columns = [x.replace('_', ' ') for x in SKU_append.columns]
    data_SKU = data_SKU[~data_SKU['SKU'].astype(str).isin(SKU_append['SKU'].astype(str))]
    data_SKU = data_SKU._append(SKU_append, ignore_index = True, sort = False)

data_SKU.loc[data_SKU['Price List NFI'].isin(['-']), 'Price List NFI'] = 0
to_excel = data_SKU.to_excel('SKU_File\data_SKU.xlsx', index = False)
print('data SKU done')

data_SKU['Price List NFI'] = data_SKU['Price List NFI'].astype(float).astype('int64')

download = True

Reading Data SKU
Scrapping Tatanama
ok
data SKU done


In [2]:
## run-2
## belum scrap dari forstok
download = False

In [ ]:
## run-3
## scrap data forstok & blibli lmen

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from datetime import datetime, timedelta
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import requests
import time
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.chrome.options import Options
import pandas as pd
import numpy as np
from webdriver_manager.chrome import ChromeDriverManager

#Set dayfirst
pd.set_option("display.date_dayfirst", True)

#ganti diff_date kalau mau panjangin periode tarikan data
diff_date = 10
data_now = str(datetime.today().day) + ' ' + str(datetime.today().strftime("%B"))

print('Input Forstok Date')
start_date = datetime.today() - timedelta(days=diff_date)
end_date = datetime.today()
date = start_date.strftime('%Y-%m-%d') + '-' + end_date.strftime('%Y-%m-%d')
print(date)

options = Options()
options.add_argument('--disable-blink-features=AutomationControlled')

#Forstok Mulai
if not download:
    print('Opening Chrome')
    driver = webdriver.Chrome()
    driver.maximize_window()
    actions = ActionChains(driver)
    driver.get("https://www.forstok.com/dashboard/users/login")
    print('Login Forstok')
    
    time.sleep(10)
    print('input email')
    username = driver.find_element(By.NAME, "email")
    username.clear()
    username.send_keys("timotius.giovandi@nutrifood.co.id")

    print('input password')
    password = driver.find_element(By.NAME, "password")
    password.send_keys("timo123")
    
    print('click')
    driver.find_element(By.XPATH, "/html/body/div[1]/div/div/div/form/input[3]").click()
    
    time.sleep(10)
    datefield = WebDriverWait(driver, 100).until(EC.element_to_be_clickable((By.XPATH,'/html/body/div[1]/section/section[2]/article/div/div/aside/div/div[1]/section[4]/div/button')))
    time.sleep(20)
    time.sleep(20)
    driver.execute_script("return arguments[0].scrollIntoView(true);", datefield)
    time.sleep(10)
    driver.find_element(By.XPATH, "/html/body/div[1]/section/section[2]/article/div/div/aside/div/div[1]/section[4]/div/button").click()
    time.sleep(10)
    driver.find_element(By.XPATH, '//*[@id="exportSalesOrderForm"]/div/section/section/div[1]/section/article/article[1]/div/section/div/div[2]').click()
    pilih=0
    while pilih < 3:    
            n_elem=0
            for i in driver.find_elements(By.XPATH, '(//*[@class="DateRangePicker__MonthHeaderLabel DateRangePicker__MonthHeaderLabel--month"])'):
                val=(driver.find_elements(By.XPATH, '(//*[@class="DateRangePicker__MonthHeaderLabel DateRangePicker__MonthHeaderLabel--month"])')[n_elem].text.split()[0])
                if val==start_date.strftime('%B'):
                    fin_n_elem=n_elem
                n_elem=n_elem+1
            if fin_n_elem==0:
                text = '(//*[@class="DateRangePicker__Month"])[1]//*[@class="DateRangePicker__DateLabel" and (text()="' + str(start_date.day) + '")]'
                if len(driver.find_elements(By.XPATH, text))==2:
                    text = '('+text+')[2]'
            else:
                text = '(//*[@class="DateRangePicker__Month"])[2]//*[@class="DateRangePicker__DateLabel" and (text()="' + str(start_date.day) + '")]'
                if len(driver.find_elements(By.XPATH, text))==2:
                    text = '('+text+')[1]'
            driver.find_element(By.XPATH, text).click()
            time.sleep(10)
    
            n_elem=0
            for i in driver.find_elements(By.XPATH, '(//*[@class="DateRangePicker__MonthHeaderLabel DateRangePicker__MonthHeaderLabel--month"])'):
                val=(driver.find_elements(By.XPATH, '(//*[@class="DateRangePicker__MonthHeaderLabel DateRangePicker__MonthHeaderLabel--month"])')[n_elem].text.split()[0])
                if val==end_date.strftime('%B'):
                    fin_n_elem=n_elem
                n_elem=n_elem+1
            if fin_n_elem==0:
                text = '(//*[@class="DateRangePicker__Month"])[1]//*[@class="DateRangePicker__DateLabel" and (text()="' + str(end_date.day) + '")]'
                if len(driver.find_elements(By.XPATH, text))==2:
                    text = '('+text+')[2]'
            else:
                text = '(//*[@class="DateRangePicker__Month"])[2]//*[@class="DateRangePicker__DateLabel" and (text()="' + str(end_date.day) + '")]'
                if end_date.strftime('%B')==start_date.strftime('%B'):
                    text = '('+text+')'#[2]
                else :
                    text = '('+text+')[1]'
            driver.find_element(By.XPATH, text).click()
            time.sleep(10)
            pilih=pilih+1
    driver.find_element(By.XPATH, '//*[@type="button" and contains(text(),"Apply")]').click()
    time.sleep(30)
    driver.find_element(By.XPATH, '/html/body/div[1]/section/section[2]/div/form/div/section/section/div[2]/aside[2]/button[2]').click()

    time.sleep(2)
    driver.quit()
   
    print('Forstok Downloaded')
    download = True

print('Download Data SKU')
#Import library

import time
import pandas as pd
import numpy as np
import math

import smtplib 
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
from email.mime.multipart import MIMEMultipart
from smtplib import SMTP
import smtplib
import sys
import requests
import os

print('Waiting to Download Forstok')
s = requests.Session()
r = s.get("https://www.forstok.com/dashboard/users/login")
data = {'dashboard_user_email' : 'andra.miftah@nutrifood.co.id', "dashboard_user_password" : 'nutrimart1234'}
r = s.post("https://www.forstok.com/dashboard/users/login", data = data)
startdate = start_date-timedelta(days=1)
startdate1 = startdate.strftime("%Y-%m-%d")
enddate = end_date.strftime("%Y-%m-%d")

text = 'https://forstok-staging-storage.s3.ap-southeast-1.amazonaws.com/items_import/Nutrifood-forstok-sales_orders-version_2-' + startdate1 + 'T17:00:00Z-' + enddate + 'T16:59:59Z.xlsx'
print(text)
while True:
   r = s.get(text)
   print('Waiting to Download Forstok')
  
   if r.status_code == requests.codes.ok:
       print(r.status_code)
       with open('Input Data/Forstok_new_input.xls', 'wb') as output:
           output.write(r.content)
       break
   else :
       time.sleep(60)
          
with open('Input Data/nutrifood--forstok-sales_orders-' + date + '.xls', 'wb') as output:
   output.write(r.content)
    
# Import data
start_time = time.time()
print("Import Data ====== 1/10")

data_forstok = pd.read_excel(r'Input Data/Forstok_new_input.xls')

#EMERGENCY DOWNLOAD IF MAINTAINANCE
# data_forstok = pd.read_excel(r"C:\Users\steven.nathanael\Downloads\Nutrifood-forstok-sales_orders-version_2-2023-06-27T17 00 00Z-2023-07-07T16 59 59Z.xlsx")
# data_forstok = data_forstok[['Order Date', 'Channel', 'Store', 'Sales Order ID',
#                             'Order Reference No.', 'Payment Status', 'Fulfillment Status', 'Payment Type',
#                             'Payment Date', 'Shipping Label Printed Date', 'Shipped Date', 'Delivered Date',
#                             'Cancelled Date', 'Return ID', 'Shipping Courier', 'Service Type', 'AWB', 'Customer Name',
#                             'Customer Email', 'Currency Code', 'Item Name', 'SKU Code', 'Bundle SKU Code', 'Barcode ID',
#                             'Warehouse Name', 'Warehouse Code', 'Quantity', 'Regular Price', 'Selling Price',
#                             'Sub Total', 'VAT', 'Shipping Fee (Non-cashless)', 'Voucher Seller', 'Platform Rebate',
#                             'Shipping Customer Name', 'Shipping Address1', 'Shipping Address2', 'Shipping City',
#                             'Shipping Zip', 'Shipping Province', 'Shipping Country', 'Shipping Phone', 'Order Note',
#                             'Invoice ID']]

data_forstok.rename(columns = {'Shipping Label Printed Date' : 'Printed Date',
                              'Shipping Date' : 'Fulfilled Date',
                              'SKU Code' : 'SKU',
                              'Shipping Fee (Non-cashless)' : 'Shipping',
                              'Voucher Seller' : 'Seller Voucher',
                              'Platform Rebate' : 'Channel Rebate',
                              'Order Note' : 'Note'}, inplace = True)

data_forstok_pure = data_forstok.copy()

data_forstok = data_forstok.dropna(how = 'all')

pd.options.mode.chained_assignment = None
# Forstok formatting
print("--- %s seconds ---" % (time.time() - start_time))
print("Formatting Data ====== 2/10")

if 'Unnamed: 35' in data_forstok:
    data_forstok = data_forstok.drop(['Unnamed: 35'], axis = 'columns')
if 'Unnamed: 36' in data_forstok:
    data_forstok = data_forstok.rename(columns={'Unnamed: 36' : 'Comment'})
if 'Unnamed: 37' in data_forstok:
    data_forstok = data_forstok.drop(['Unnamed: 37'], axis = 'columns')
if 'Unnamed: 38' in data_forstok:
    data_forstok = data_forstok.drop(['Unnamed: 38'], axis = 'columns')
if 'Unnamed: 39' in data_forstok:
    data_forstok = data_forstok.drop(['Unnamed: 39'], axis = 'columns')
if 'Unnamed: 40' in data_forstok:
    data_forstok = data_forstok.drop(['Unnamed: 40'], axis = 'columns')
if 'Unnamed: 41' in data_forstok:
    data_forstok = data_forstok.drop(['Unnamed: 41'], axis = 'columns')
    
data_forstok["Order Date"] = data_forstok["Order Date"].str.replace(' WIB', '')
data_forstok["Payment Date"] = data_forstok["Payment Date"].str.replace(' WIB', '')
data_forstok["Order Date"] = pd.to_datetime(data_forstok["Order Date"],  format="%d/%m/%Y %H:%M")
data_forstok["Payment Date"] = pd.to_datetime(data_forstok["Payment Date"], format="%d/%m/%Y %H:%M")
data_forstok["Cancelled Date"] = pd.to_datetime(data_forstok["Cancelled Date"], format="%d/%m/%Y %H:%M")
data_forstok['Customer Name'] = data_forstok['Customer Name'].fillna(data_forstok['Shipping Customer Name'])
data_forstok = data_forstok.rename(columns = {'Seller Voucher' : 'Seller Discount'})
data_forstok = data_forstok[~((data_forstok['Store']=='Shopee')&(data_forstok['Order Date']<'21-12-2021'))] ###TMO###
print('Date formatted')

# Phone formatting

data_forstok['Shipping Phone'] = data_forstok['Shipping Phone'].astype(str).str.replace('=','', regex = False)
data_forstok['Shipping Phone'] = data_forstok['Shipping Phone'].astype(str).str.replace('"','', regex = False)
data_forstok['Shipping Phone'] = data_forstok['Shipping Phone'].astype(str).str.replace('(','', regex = False)
data_forstok['Shipping Phone'] = data_forstok['Shipping Phone'].astype(str).str.replace(')','', regex = False)
data_forstok['Shipping Phone'] = data_forstok['Shipping Phone'].astype(str).str.replace('-','', regex = False)
data_forstok['Shipping Phone'] = data_forstok['Shipping Phone'].astype(str).str.replace('+','', regex = False)
data_forstok['Shipping Phone'] = data_forstok['Shipping Phone'].astype(str).str.replace('^620','0', regex = True)
data_forstok['Shipping Phone'] = data_forstok['Shipping Phone'].astype(str).str.replace('^62','0', regex = True)
data_forstok['Shipping Phone'] = data_forstok['Shipping Phone'].astype(str).str.replace('^620','0', regex = True)
data_forstok['Shipping Phone'] = data_forstok['Shipping Phone'].astype(str).str.replace('^8','08', regex = True)
data_forstok['Shipping Phone'] = data_forstok['Shipping Phone'].astype(str).str.replace('^21','021', regex = True)
data_forstok['Shipping Phone'] = data_forstok['Shipping Phone'].astype(str).str.replace('^008','08', regex = True)
data_forstok['Shipping Phone'] = data_forstok['Shipping Phone'].astype(str).str.replace('(021)','021', regex = False)
data_forstok['Shipping Phone'] = data_forstok['Shipping Phone'].astype(str).str.replace('+62','0', regex = False)
print('Phone Formatted')

# Forstok SKU
print("--- %s seconds ---" % (time.time() - start_time))
print("Fulfilling SKU ====== 3/10")
indeks = data_forstok[data_forstok['Item Name'].astype(str) == 'Buy 1 Get 1 FREE Tropicana Slim Goldenmil Vanilla Manuka Honey (6 Sch)'][data_forstok[data_forstok['Item Name'].astype(str) == 'Buy 1 Get 1 FREE Tropicana Slim Goldenmil Vanilla Manuka Honey (6 Sch)']['SKU'] == 'PE8B27'].index.to_list()
data_forstok['SKU'][indeks] = '2101384106P2'
data_forstok.loc[data_forstok['SKU']=="(E)2101492P04",'SKU']="(E)2101492P4"
data_forstok.loc[data_forstok['SKU']=="P2101151180P2",'SKU']="PT7(2)B65"
data_forstok.loc[data_forstok['SKU']==' PT28(2)B65','SKU']='PT28(2)B65'
data_forstok.loc[data_forstok['SKU']==' PT6(2)B65','SKU']='PT6(2)B65'
data_forstok.loc[data_forstok['SKU']==' PT1(2)B65','SKU']='PT1(2)B65'
data_forstok.loc[(data_forstok['SKU']=='2101865450P3')&(data_forstok['Item Name'].astype(str)=='Tropicana Slim White Coffee 4 Sachet x 3 pcs - Kopi Susu Nikmat Tanpa Gula Pasir'),'SKU']='2104170164P3'
data_forstok.loc[(data_forstok['SKU']=='2104170104P3')&(data_forstok['Item Name'].astype(str)=='Triple Pack: Tropicana Slim White Coffee - Kopi Bebas Gula'),'SKU']='2104170164P3'
data_forstok.loc[(data_forstok['SKU']=='COLLETTEP24')&(data_forstok['Item Name'].astype(str)=='HiLo School Cotton Candy 200ml - Ready to Drink (24 Pcs)'),'SKU']='2HF1403250P24'
data_forstok.loc[(data_forstok['SKU']=='COLLETTEP4')&(data_forstok['Item Name'].astype(str)=='HiLo School Cotton Candy 200ml - Ready to Drink (4 Pcs) Tinggi Kalsium'),'SKU']='2HF1403250P4'

skushopee = data_forstok[data_forstok['SKU'].astype(str).str.contains('(S)',regex = False)]
data_forstok = data_forstok[~data_forstok['SKU'].astype(str).isin(skushopee['SKU'].astype(str))]

skushopee = skushopee.reset_index(drop = True)
data_forstok = data_forstok.reset_index(drop = True)

forstok_all_sku = pd.read_excel(r'SKU_File\forstok_all_sku.xlsx')
indeks = data_forstok[data_forstok['SKU'].isnull()].index.to_list()

for i in indeks:
    if str(data_forstok['Item Name'][i]).lower() in data_SKU['Nama Produk'].astype(str).str.lower().values:
        data_forstok['SKU'][i] = data_SKU['SKU'].loc[str(data_forstok['Item Name'][i]).lower() == data_SKU['Nama Produk'].astype(str).str.lower()].values[0]

indeks = data_forstok[data_forstok['SKU'].isnull()].index.to_list()
for i in indeks:
    if str(data_forstok['Item Name'][i]).lower() in forstok_all_sku['Item Name'].astype(str).str.lower().values:
        data_forstok['SKU'][i] = forstok_all_sku['SKU'].loc[str(data_forstok['Item Name'][i]).lower() == forstok_all_sku['Item Name'].astype(str).str.lower()].values[0]

indeks = data_forstok[data_forstok['Item Name'].astype(str).str.contains(' - ')].index.to_list()

# Formatting double name
for i in indeks:
    if data_forstok['Item Name'][i].count(' - ') == 1 :
        if (data_forstok['Item Name'][i].split(' - ')[0] == data_forstok['Item Name'][i].split(' - ')[1]):
            data_forstok['Item Name'][i] = data_forstok['Item Name'][i].split(' - ')[0]
    elif data_forstok['Item Name'][i].count(' - ') > 1:
        temp = math.ceil(data_forstok['Item Name'][i].count(' - ')/2)
        itemname = ''
        duplicate = ''
        for j in range(temp):
            if j == 0:
                itemname = itemname + data_forstok['Item Name'][i].split(' - ')[j]
                duplicate = duplicate + data_forstok['Item Name'][i].split(' - ')[temp+j]
            else :
                itemname = itemname + ' ' +data_forstok['Item Name'][i].split(' - ')[j]
                duplicate = duplicate + ' ' +data_forstok['Item Name'][i].split(' - ')[temp+j]
        if itemname == duplicate:
            data_forstok['Item Name'][i] = itemname
        
data_forstok['Item Name'] = data_forstok['Item Name'].str.replace(r' - $', '')
data_forstok["Item Name"] = data_forstok["Item Name"].str.replace('Special Promo by Nutrimart Serba 12 RIBU Varian:', '', regex=False)
data_forstok["Item Name"] = data_forstok["Item Name"].str.replace('Khusus Jabodetabek', '- Khusus Jabodetabek', regex=False)
data_forstok['Item Name'] = data_forstok['Item Name'].str.replace('Buy 1 Get F', 'Buy 1 Get 1 F')
data_forstok['Item Name'] = data_forstok['Item Name'].str.replace('Buy 1 Get H', 'Buy 1 Get 1 H')
data_forstok['Item Name'] = data_forstok['Item Name'].str.replace('Buy 12 FREE', 'Buy 12 FREE 12')

# Formatting SKU based on name
indeks = data_forstok[data_forstok['SKU'].isnull()].index.to_list()
for i in indeks:
    if str(data_forstok['Item Name'][i]).lower().strip() in data_SKU['Nama Produk'].astype(str).str.lower().str.strip().values:
        data_forstok['SKU'][i] = data_SKU['SKU'].loc[str(data_forstok['Item Name'][i]).lower().strip() == data_SKU['Nama Produk'].astype(str).str.lower().str.strip()].values[0]

indeks = data_forstok[data_forstok['SKU'].isnull()].index.to_list()
for i in indeks:
    if str(data_forstok['Item Name'][i]).lower() in forstok_all_sku['Item Name'].astype(str).str.lower().values:
        data_forstok['SKU'][i] = forstok_all_sku['SKU'].loc[str(data_forstok['Item Name'][i]).lower() == forstok_all_sku['Item Name'].astype(str).str.lower()].values[0]

indeks = data_forstok[data_forstok['SKU'].isnull()].index.tolist()

data_adasku = data_forstok[['Item Name', 'SKU']]
data_adasku = data_adasku[data_adasku['SKU'].notnull()]

data_nosku = data_forstok[['Item Name', 'SKU']]
data_nosku = data_nosku[data_nosku['SKU'].isnull()]

for i in indeks:
    if data_adasku['SKU'].loc[data_nosku['Item Name'][i] == data_adasku['Item Name']].size != 0:
        data_forstok['SKU'][i] = data_adasku['SKU'].loc[data_nosku['Item Name'][i] == data_adasku['Item Name']].values[0]

for i in indeks:
    if data_forstok['Item Name'][i] == 'Lokalate Kopi Durian 10s':
        data_forstok['SKU'][i] = '1101675318'
    elif data_forstok['Item Name'][i] == 'Nutrisari Madu Kurma Isi 16 Renceng X 10 Sachet Karton' or data_forstok['Item Name'][i] =='Nutrisari Madu Kurma Isi 16 Renceng X 10 SachetKarton': 
        data_forstok['SKU'][i] = 'PN30(16)'
    elif data_forstok['Item Name'][i] == 'L-Men Protein Bar Crunchy Chocolate Isi X12 (Exp Date:10-Apr-2019)':
        data_forstok['SKU'][i] = '2306592173'
    elif data_forstok['Item Name'][i] == 'FS Hilo Active Chocolate Minuman Kesehatan [750 gr]' or data_forstok['Item Name'][i] == 'FSHilo Active Chocolate Minuman Kesehatan [750 gr]':
        data_forstok['SKU'][i] = '2101452190'
    elif data_forstok['Item Name'][i] == 'FS L-Men Platinum Suplemen Kesehatan + Free Spider Bottle [800 g] Hitam' or data_forstok['Item Name'][i] == 'FSL-Men Platinum Suplemen Kesehatan + Free Spider Bottle [800 g] Hitam':
        data_forstok['SKU'][i] = '2305551288P1G26'
    elif data_forstok['Item Name'][i] == 'NutriSari Premium ala Jus Mangga':
        data_forstok['SKU'][i] = '1100534104'
    elif data_forstok['Item Name'][i] == 'Buy 1 Get 1 FREE Tropicana Slim Sweetener Honey (50 Sch) - FS':
        data_forstok['SKU'][i] = '2102501125P1G53'

indeks = data_forstok[~data_forstok['SKU'].astype(str).isin(data_SKU['SKU'].astype(str))].index.to_list()
for i in indeks:
    if str(data_forstok['Item Name'][i]).lower() in forstok_all_sku['Item Name'].astype(str).str.lower().values:
        data_forstok['SKU'][i] = forstok_all_sku['SKU'].loc[str(data_forstok['Item Name'][i]).lower() == forstok_all_sku['Item Name'].astype(str).str.lower()].values[0]

list_alias = []
list_alias_name = []
for colname in data_SKU.columns:
    if 'Alias SKU' in colname:
        list_alias.append(colname)
    if 'Alias Nama' in colname:
        list_alias_name.append(colname)


for i in indeks:
    for j in list_alias:
        if str(data_forstok['SKU'][i]) in data_SKU[j].astype(str).values:
            idx = data_SKU[str(data_forstok['SKU'][i]) == data_SKU[j].astype(str)].index.to_list()
            for k in idx:
                if str(data_forstok['Item Name'][i]) == data_SKU[j.replace('SKU', 'Nama')][k]:
                    data_forstok['SKU'][i] = data_SKU['SKU'][k]
                        
indeks = data_forstok[~data_forstok['SKU'].astype(str).isin(data_SKU['SKU'].astype(str))].index.to_list()

for i in indeks:
    for j in list_alias_name:
        if str(data_forstok['Item Name'][i]).lower() in data_SKU[j].astype(str).str.lower().values:
            data_forstok['SKU'][i] = data_SKU['SKU'].loc[str(data_forstok['Item Name'][i]).lower() == data_SKU[j].astype(str).str.lower()].values[0]

indeks = data_forstok[~data_forstok['SKU'].astype(str).isin(data_SKU['SKU'].astype(str))].index.to_list()

for i in indeks:
    if str(data_forstok['Item Name'][i]).lower() in data_SKU['Nama Produk'].astype(str).str.lower().values:
        data_forstok['SKU'][i] = data_SKU['SKU'].loc[str(data_forstok['Item Name'][i]).lower() == data_SKU['Nama Produk'].astype(str).str.lower()].values[0]

indeks = skushopee[~skushopee['SKU'].astype(str).str.replace('(S)','', regex = False).isin(data_SKU['SKU'].astype(str))].index.to_list()
for i in indeks:
    if str(skushopee['Item Name'][i]).lower() in data_SKU['Nama Produk'].astype(str).str.lower().values:
        skushopee['SKU'][i] = data_SKU['SKU'].loc[str(skushopee['Item Name'][i]).lower() == data_SKU['Nama Produk'].astype(str).str.lower()].values[0]
        skushopee['SKU'][i] = '(S)' + str(skushopee['SKU'][i])

indeks = skushopee[~skushopee['SKU'].astype(str).str.replace('(S)','', regex = False).isin(data_SKU['SKU'].astype(str))].index.to_list()
for i in indeks:
    for j in list_alias:
        if str(skushopee['SKU'][i]).replace('(S)','') in data_SKU[j].astype(str).values:
            idx = data_SKU[str(skushopee['SKU'][i]).replace('(S)','') == data_SKU[j].astype(str)].index.to_list()
            for k in idx:
                if str(skushopee['Item Name'][i]) == data_SKU[j.replace('SKU', 'Nama')][k]:
                    skushopee['SKU'][i] = data_SKU['SKU'][k]

indeks = data_forstok[data_forstok['SKU'].astype(str) == 'Gift Sosro'].index.to_list()
data_forstok = data_forstok.drop(indeks, axis = 0).reset_index(drop = True)

indeks = data_forstok[data_forstok['Item Name'].astype(str).str.contains('Fricella')].index.to_list()
data_forstok = data_forstok.drop(indeks, axis = 0).reset_index(drop = True)

indeks = data_forstok[data_forstok['Item Name'].astype(str).str.contains('Zaskia Mecca')].index.to_list()
data_forstok = data_forstok.drop(indeks, axis = 0).reset_index(drop = True)

print("--- %s seconds ---" % (time.time() - start_time))
print("Listing SKU Missing ====== 4/10")

data_forstok = data_forstok._append(skushopee, ignore_index = True, sort = False)
data_forstok = data_forstok.reset_index(drop = True)

indeks = data_forstok[data_forstok['SKU'].astype(str) == '71210111'].index.to_list()
data_forstok['SKU'][indeks] = 'PN28N29N34(2)N53N54N55'

indeks = data_forstok[data_forstok['SKU'].astype(str) == '71210112'].index.to_list()
data_forstok['SKU'][indeks] = 'PN19N22N30N34(2)N53N55'

indeks = data_forstok[data_forstok['SKU'].astype(str) == '(B) 2101656'].index.to_list()
data_forstok['SKU'][indeks] = '(B)2101656'

indeks = data_forstok[data_forstok['Item Name'].astype(str) == 'PRE ORDER NutriSari NUTRI-C1000 40sch Suplemen Kesehatan Vit C 1000mg '].index.to_list()
data_forstok['SKU'][indeks] = '1102110453'

data_forstok.loc[data_forstok['Item Name']=="HiLo Active Caramel Latte 500gr - Susu Tinggi Kalsium",'SKU']="2101478180"
data_forstok.loc[data_forstok['Item Name']=="NutriSari Es Rujak 40 Sachet - Minuman Buah Vitamin C - Expired 4 Bulan",'SKU']="(E)1101984453"
data_forstok.loc[data_forstok['SKU']=='PH39(12)U8(12) ','SKU']='PH39(12)U8(12)'


shopee_con = pd.read_excel(r'data_supp\Shopee Converter.xlsx')
indeks = data_forstok[data_forstok['SKU'].astype(str).str.replace('(S)','', regex = False).isin(shopee_con['SKU'].astype(str))][data_forstok[data_forstok['SKU'].astype(str).str.replace('(S)','', regex = False).isin(shopee_con['SKU'].astype(str))]['Item Name'].astype(str).str.lower().isin(shopee_con['Product Name'].astype(str).str.lower())].index.to_list()
product = data_forstok[data_forstok['SKU'].astype(str).str.replace('(S)','', regex = False).isin(shopee_con['SKU'].astype(str))][data_forstok[data_forstok['SKU'].astype(str).str.replace('(S)','', regex = False).isin(shopee_con['SKU'].astype(str))]['Item Name'].astype(str).str.lower().isin(shopee_con['Product Name'].astype(str).str.lower())]['Item Name'].unique()

for i in product:
    indeks_shopee = data_forstok.iloc[indeks][data_forstok.iloc[indeks]['Item Name'].astype(str).str.lower() == str(i).lower()].index.to_list()
    data_forstok['SKU'][indeks_shopee] = shopee_con[shopee_con['Product Name'].astype(str).str.lower() == str(i).lower()]['Real SKU'].values[0]


to_excel = data_forstok.to_excel(r'forstok_new.xlsx', index = False)

skushopee = data_forstok[data_forstok['SKU'].astype(str).str.contains('(S)',regex = False)]
data_forstok = data_forstok[~data_forstok['SKU'].astype(str).isin(skushopee['SKU'].astype(str))]

skushopee = skushopee.reset_index(drop = True)
data_forstok = data_forstok.reset_index(drop = True)

sku_exclude = ['5337754001-1623228299159-0',"TESSS"]
data_forstok = data_forstok[~data_forstok['SKU'].isin(sku_exclude)]

data_forstok.loc[data_forstok['Item Name']=="HiLo Teen Strawberry Milkshake 500 gram – Susu Tinggi Kalsium - Milkshake",'SKU']="2101683180"
data_forstok.loc[data_forstok['Item Name']=="Paket Minyak Hemat – Tropicana Slim Canola Oil & Tropicana Slim Sunflower Oil - Package",'SKU']="PT3T46"
data_forstok.loc[data_forstok['Item Name']=="Buy 2 Get 1 - L-Men Protein Crunch BBQ Beef (20gr) - Triplepack",'SKU']="2309005300P3"
data_forstok.loc[data_forstok['Item Name']=="Paket Kecap Sehat - Topicana Slim Kecap Manis & Tropicana Slim Kecap Asin - Package","SKU"]="PT26T44"
data_forstok.loc[(data_forstok['Item Name']=="(Free Gift) 3pcs L-Men Protein Crunch BBQ Beef 20g")&
                 (data_forstok['Store']=="Lazada"),"SKU"]="(B)2309005305P3"
data_forstok.loc[data_forstok['Item Name']=="WHS - Paket NutriSari Jeruk Peras, HiLo Es Ketan Hitam, dan HiLo Klepon Latte Powder Drink Refill 500 gram",'SKU']="PH81H82N60G187"
data_forstok.loc[data_forstok['Item Name']=="WHS - HiLo Milky Brown Sugar Ready to Drink 200ml (24 tetrapack) - Minuman Sumber Kalsium Rendah Lemak",'SKU']="2101941250P24"
data_forstok.loc[data_forstok['Item Name']=="WHS - Twin Pack - Tropicana Slim Shirataki Noodle: Mie Goreng [71 g]",'SKU']="2106023014"
data_forstok.loc[data_forstok['Item Name']=="WHS - HiLo School Chocolate Ready to Drink 200ml (4 tetrapack) - Susu Tinggi Kalsium Rendah Lemak",'SKU']="2101492P4"
data_forstok.loc[data_forstok['Item Name']=="WHS - HiLo Kacang Hijau Ready to Drink 200ml (24 tetrapack) - Minuman Sumber Kalsium Rendah Lemak",'SKU']="2101907250P24"
data_forstok.loc[data_forstok['Item Name']=="WHS NutriSari Es Rujak Jeruk Bali 40sch Minuman Buah Vitamin C Vitamin D",'SKU']="1101996453"
data_forstok.loc[data_forstok['Item Name']=="DISKON BOMBASTIS - Tropicana Slim Almond Drink Chocolicious 190ml (1 pcs) - Minuman Almond Ready to Drink Rasa Cokelat ",'SKU']="2T01402249"
data_forstok.loc[data_forstok['Item Name']=="DISKON BOMBASTIS Tropicana Slim Almond Drink Chocolicious 190ml (1 pcs) Minuman Almond Ready to Drink Rasa Cokelat ",'SKU']="2T01402249"
data_forstok.loc[data_forstok['Item Name']=="WHS NutriSari RTD Squeezed Orange Minuman Jeruk Peras Vitamin C 200 mL ",'SKU']="1N01402250P4"
data_forstok.loc[data_forstok['Item Name']=="WHS HiLo Active Vanilla 500g x 2 pcs Susu Tinggi Kalsium Rendah Lemak",'SKU']="2101485180P2"
data_forstok.loc[data_forstok['Item Name']=="WHS HiLo Teen Taro 500 gr Susu Tinggi Kalsium",'SKU']="2101609180"
data_forstok.loc[data_forstok['Item Name']=="Tropicana Slim Hampers Lebaran Idul Fitri Fancy Tote Bag",'SKU']="PT24T51T52T61T79G292G294"
data_forstok.loc[data_forstok['Item Name']=="WHS HiLo Chocolate Avocado Ready to Drink 200ml (4 Tetrapack) Susu Siap Minum Tinggi Kalsium",'SKU']="(E)2HH1407250P4"
data_forstok.loc[data_forstok['Item Name']=="TG HiLo School Chocolate RTD 200ml",'SKU']="2101492"
data_forstok.loc[data_forstok['Item Name']=="TG HiLo School Chocolate RTD 200ml ",'SKU']="2101492"
data_forstok.loc[data_forstok['Item Name']=="TG Tropicana Slim Almond Drink Chocolicious 190ml (1 pcs) Minuman Almond Ready to Drink Rasa Cokelat ",'SKU']="2T01402249"
data_forstok.loc[data_forstok['Item Name']=="Tropicana Slim Oat Drink Japanese Cantaloupe Melon 190 ml (1 Pcs) Minuman Oat Vegan Plant Based Bebas Gula Ready to Drink",'SKU']="2T01403249"
data_forstok.loc[data_forstok['SKU']=="1101588454",'SKU']="1101588453"
data_forstok.loc[data_forstok['Item Name']=="WHS HiLo Gold Biscuit Cereal 500g Susu Tinggi Kalsium Lebih Rendah Lemak",'SKU']="(E)2101524180"
data_forstok.loc[data_forstok['Item Name']=="Nutrifood BBF Mystery Box",'SKU']="PH100H108(2)T71"
data_forstok.loc[data_forstok['SKU']=="(E)2104406P1",'SKU']="(E)2104406"
data_forstok.loc[data_forstok['Item Name']=="WHS HiLo Teen Chocolate Ready to Drink (4 pcs) Susu Tinggi Kalsium Rendah Lemak",'SKU']="(E)2101461P4"
data_forstok.loc[data_forstok['Item Name']=="WHS Paket HiLo School HiLo School Vanilla Vegiberi 500g & Original 400g",'SKU']="PH4H102"
data_forstok.loc[data_forstok['Item Name']=="WHS Twin Pack HiLo Gold Biscuit Cereal 500g Susu Tinggi Kalsium Lebih Rendah Lemak",'SKU']="(E)2101524180P2"
data_forstok.loc[data_forstok['Item Name']=="WHS Paket School 3+ HiLo School 3+ Strawberry Pop 400g & School 3+ Soya Vanilla Malt 400g",'SKU']="PE229E230"
data_forstok.loc[data_forstok['Item Name']=="WHS HiLo Gold Susu Tinggi Kalsium Rendah Lemak Chocolate [500 g ]",'SKU']="(E)2101551180"
data_forstok.loc[data_forstok['Item Name']=="TG HiLo Protein Chocofit UHT 190 mL (1 pcs) – Susu Tinggi Protein dengan Collagen Rasa Cokelat",'SKU']="2HH1409249"

idx = []
idx = idx + data_forstok[data_forstok['SKU'].isnull()].drop_duplicates().index.to_list()
idx = idx + data_forstok[~data_forstok['SKU'].astype(str).isin(data_SKU['SKU'].astype(str))].index.to_list()
idx = list(dict.fromkeys(idx))
idx_s = skushopee[~skushopee['SKU'].astype(str).str.replace('(S)','', regex = False).isin(data_SKU['SKU'].astype(str))].index.to_list()
idx_s = list(dict.fromkeys(idx_s))


for i in product:
    indeks_shopee = data_forstok.iloc[indeks][data_forstok.iloc[indeks]['Item Name'].astype(str).str.lower() == str(i).lower()].index.to_list()
    data_forstok['SKU'][indeks_shopee] = shopee_con[shopee_con['Product Name'].astype(str).str.lower() == str(i).lower()]['Real SKU'].values[0]

print("--- %s seconds ---" % (time.time() - start_time))
if len(idx) != 0 or len(idx_s) != 0:
    alert = data_forstok.iloc[idx, ][['SKU', 'Item Name', 'Channel']].drop_duplicates()
    alert = alert._append(skushopee.iloc[idx_s][['SKU', 'Item Name', 'Channel', 'Selling Price']].drop_duplicates(), ignore_index = True, sort = False)
    alert['SKU'] = alert['SKU'].astype(str)
    alert['SKU Valid'] = np.nan
    to_excel = alert.to_excel('ALERT_FORSTOK_SKU_MISSING.xlsx')
    print("Some SKU Missing Please Complete It ====== 5/10")
    print("--- %s seconds ---" % (time.time() - start_time))
    
    # creates SMTP session 
    s = smtplib.SMTP('smtp.gmail.com', 587) 

    # start TLS for security 
    s.starttls() 

    # Authentication 
    s.login("automationnfi@gmail.com", "nutrifood2020") 

    msg = MIMEMultipart()
    msg['Subject'] = "ALERT SKU FORSTOK MISSING"


    html = """\
    <html>
      <head></head>
      <body>
        {0}
      </body>
    </html>
    """.format(alert.to_html())

    part1 = MIMEText(html, 'html')
    msg.attach(part1)

    # sending the mail 
    s.sendmail("automationnfi@gmail.com", "steven.nathanael@nutrifood.co.id", msg.as_string()) 

    # terminating the session 
    s.quit() 
else :
    print("Filling Brand ====== 5/10")
    data_forstok['SKU'] = data_forstok['SKU'].astype(str)
    data_forstok['Item Name'] = data_forstok['Item Name'].astype(str)
    data_SKU['Real SKU'] = data_SKU['SKU'].astype(str).str.replace('(S)', '', regex = False)
    data_SKU['Real Nama Produk'] = data_SKU['Nama Produk'].astype(str)
    
    data_forstok = data_forstok.merge(data_SKU[['Real SKU', 'Real Nama Produk']].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU')

    temp = data_forstok[data_forstok['Real SKU'].isnull()].copy()
    temp['SKU'] = temp['SKU'].astype(str).str.replace('(S)','', regex = False)
    temp = temp.merge(data_SKU[['Real SKU', 'Real Nama Produk']].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU').set_index(temp.index)
    temp['Real SKU_x'] = temp['Real SKU_x'].fillna(temp['Real SKU_y'])
    temp['Real Nama Produk_x'] = temp['Real Nama Produk_x'].fillna(temp['Real Nama Produk_y'])
    temp = temp.drop(['Real SKU_y', 'Real Nama Produk_y'], axis = 1)
    temp = temp.rename(columns = {'Real SKU_x' : 'Real SKU', 'Real Nama Produk_x' : 'Real Nama Produk'})

    indeks = data_forstok[data_forstok['Real SKU'].isnull()].index.to_list()
    data_forstok['Real SKU'][indeks] = temp['Real SKU'][indeks]
    data_forstok['Real Nama Produk'][indeks] = temp['Real Nama Produk'][indeks]

    temp = data_forstok[data_forstok['Real SKU'].isnull()].copy()
    temp['SKU'] = temp['SKU'].astype(str).str.replace('hd','', regex = False)
    temp['SKU'] = temp['SKU'].astype(str).str.replace('HD','', regex = False)
    temp = temp.merge(data_SKU[['Real SKU', 'Real Nama Produk']].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU').set_index(temp.index)
    temp['Real SKU_x'] = temp['Real SKU_x'].fillna(temp['Real SKU_y'])
    temp['Real Nama Produk_x'] = temp['Real Nama Produk_x'].fillna(temp['Real Nama Produk_y'])
    temp = temp.drop(['Real SKU_y', 'Real Nama Produk_y'], axis = 1)
    temp = temp.rename(columns = {'Real SKU_x' : 'Real SKU', 'Real Nama Produk_x' : 'Real Nama Produk'})

    indeks = data_forstok[data_forstok['Real SKU'].isnull()].index.to_list()
    data_forstok['Real SKU'][indeks] = temp['Real SKU'][indeks]
    data_forstok['Real Nama Produk'][indeks] = temp['Real Nama Produk'][indeks]

    data_forstok['Real SKU'] = data_forstok['Real SKU'].astype(str)
    data_forstok = data_forstok.merge(data_SKU[['SKU', 'Brand', 'Sub Brand', 'Parent Item', 'Parent SKU']].drop_duplicates(['SKU']), how = 'left', left_on = 'Real SKU', right_on = 'SKU')
    data_forstok = data_forstok.drop(['SKU_y'], axis = 1)
    data_forstok = data_forstok.rename(columns = {'SKU_x':'SKU'})

    print("--- %s seconds ---" % (time.time() - start_time))
    print("Unbundling ====== 6/10")        
    # Forstok Unbundling    
    list_col = ['SKU'] + data_SKU.columns[data_SKU.columns.get_loc('Produk 1'):data_SKU.columns.get_loc('Harga Organik 7')+1].to_list()
    data_forstok = data_forstok.merge(data_SKU[list_col].drop_duplicates(['SKU']), how = 'left', left_on = 'Real SKU', right_on = 'SKU')
    list_pcs = [x for x in data_forstok.columns if 'PCS' in x]
    for i in list_pcs:
        data_forstok[i] = data_forstok[i] * data_forstok['Quantity']
    data_forstok = data_forstok.drop(['SKU_y'], axis = 1)
    data_forstok = data_forstok.rename(columns = {'SKU_x':'SKU'})

    indeks = data_forstok[data_forstok['Brand'] == 'Bundle'].index.to_list()
    data_forstok['Bundle Flag'] = np.nan
    data_forstok['Bundle Flag'][indeks] = 'Bundle'

    indeks = data_forstok[data_forstok['Brand'] == 'Bundle'][data_forstok[data_forstok['Brand'] == 'Bundle']['SKU'].astype(str).str.contains('(S)', regex = False)].index.to_list()
    data_forstok['SKU Produk 1'][indeks] = '(S)' + data_forstok['SKU Produk 1'][indeks].astype(str)
    data_forstok['SKU Produk 2'][indeks] = '(S)' + data_forstok['SKU Produk 2'][indeks].astype(str)
    data_forstok['SKU Produk 3'][indeks] = '(S)' + data_forstok['SKU Produk 3'][indeks].astype(str)
    data_forstok['SKU Produk 4'][indeks] = '(S)' + data_forstok['SKU Produk 4'][indeks].astype(str)
    data_forstok['SKU Produk 5'][indeks] = '(S)' + data_forstok['SKU Produk 5'][indeks].astype(str)
    data_forstok['SKU Produk 6'][indeks] = '(S)' + data_forstok['SKU Produk 6'][indeks].astype(str)
    data_forstok['SKU Produk 7'][indeks] = '(S)' + data_forstok['SKU Produk 7'][indeks].astype(str)

    print("--- %s seconds ---" % (time.time() - start_time))
    print("Filling Date ====== 7/10")
    data_forstok['Date'] = np.nan
    data_forstok['Month'] = np.nan
    data_forstok['Year'] = np.nan

    for i in range(data_forstok.shape[0]):
        if int(data_forstok['Order Date'][i].strftime('%d')) <= 12:
            data_forstok['Date'][i] = pd.to_datetime(data_forstok['Order Date'][i].strftime('%Y-%d-%m %H:%M')).day
            data_forstok['Month'][i] = pd.to_datetime(data_forstok['Order Date'][i].strftime('%Y-%d-%m %H:%M')).month_name()
            data_forstok['Year'][i] = pd.to_datetime(data_forstok['Order Date'][i].strftime('%Y-%d-%m %H:%M')).year
        else :
            data_forstok['Date'][i] = pd.to_datetime(data_forstok['Order Date'][i]).day
            data_forstok['Month'][i] = pd.to_datetime(data_forstok['Order Date'][i]).month_name()
            data_forstok['Year'][i] = pd.to_datetime(data_forstok['Order Date'][i]).year

    quarter = pd.DataFrame([['January', 1], ['February', 1], ['March', 1], ['April', 2], ['May', 2], ['June', 2], 
            ['July', 3], ['August', 3], ['September', 3],['October', 4], ['November', 4], ['December', 4]], columns = ['Bulan', 'Quarter'])
    data_forstok = data_forstok.merge(quarter, how = 'left', left_on = 'Month', right_on = 'Bulan')
    data_forstok = data_forstok.drop(['Bulan'], axis = 1)
    data_bulan = pd.DataFrame([{'Bulan' : 'December', 'Number' : 12} ,
            {'Bulan' : 'January' , 'Number': 1},
            {'Bulan' : 'February' , 'Number': 2},
            {'Bulan' : 'March' , 'Number': 3},
            {'Bulan' : 'April' , 'Number': 4},
            {'Bulan' : 'May' , 'Number': 5},
            {'Bulan' : 'June', 'Number': 6},
            {'Bulan' : 'July' , 'Number': 7},
            {'Bulan' : 'August', 'Number' : 8},
            {'Bulan' : 'September', 'Number' : 9},
            {'Bulan' : 'October' , 'Number': 10},
            {'Bulan' : 'November' , 'Number': 11}])
    temp = data_forstok.copy()
    temp['Day'] = temp['Date']
    temp = temp.merge(data_bulan, how = 'left', left_on = 'Month', right_on='Bulan')
    temp= temp.rename(columns = {'Month' : 'Bulan', 'Number' : 'Month'})
    data_forstok['Week'] = pd.to_datetime(temp[['Year', 'Month', 'Day']]).dt.strftime("%U")
    temp['Hour'] = pd.to_datetime(data_forstok['Order Date']).dt.hour
    temp['Minute'] = pd.to_datetime(data_forstok['Order Date']).dt.minute
    temp['Second'] = pd.to_datetime(data_forstok['Order Date']).dt.second
    data_forstok['True datetime'] = pd.to_datetime(temp[['Year', 'Month', 'Day', 'Hour', 'Minute', 'Second']])
    
    
    forstok_all = data_forstok
    forstok_all['Total'] = forstok_all['Sub Total']
    forstok_all['Price List NFI'] = np.nan
    forstok_all['Total Net'] = np.nan
    
    forstok_all.drop(columns = 'Order Status', inplace = True)

    forstok_all = forstok_all.rename(columns={'Order Reference No.' : 'Order #',
                                            'Fulfillment Status' : 'Order Status',
                                            'Order Date' : 'Order date',
                                            'Item Name' :'Product Name',
                                            'Bundle Name' : 'Bundle',
                                            'Shipping Country' : 'Country',
                                            'Shipping Province' : 'Region',
                                            'Shipping City' : 'City',
                                            'Shipping Zip' : 'Zip Code',
                                            'Shipping Address1' : 'Address',
                                            'Shipping Phone' : 'Phone',
                                            'Quantity' : 'Qty. Invoiced',
                                            'Sub Total' : 'Subtotal'})
    forstok_all['Kecamatan'] = np.nan
    forstok_all['Kelurahan'] = np.nan
    forstok_all['Store'] = forstok_all['Channel']

    print("Filling Location ===== 7/10")
    indeks = forstok_all[forstok_all['City'].astype(str).str.contains('/')]['City'].index.to_list()
    if len(indeks)>0:
        forstok_all['Kecamatan'][indeks] = forstok_all['City'][indeks].str.split('/', n = 1,expand = True)[1]
        forstok_all['City'][indeks] = forstok_all['City'][indeks].str.split('/', n = 1,expand = True)[0]

    indeks = forstok_all[forstok_all['Kecamatan'].astype(str).str.contains('-')]['Kecamatan'].index.to_list()
    if len(indeks)>0:
        forstok_all['Kelurahan'][indeks] = forstok_all['Kecamatan'][indeks].str.split('-', n = 1,expand = True)[1]
        forstok_all['Kecamatan'][indeks] = forstok_all['Kecamatan'][indeks].str.split('-', n = 1,expand = True)[0]

    indeks = forstok_all[forstok_all['City'].astype(str).str.contains(',')]['City'].index.to_list()
    if len(indeks)>0:
        forstok_all['Kecamatan'][indeks] = forstok_all['City'][indeks].str.split(',', n = 1,expand = True)[1]
        forstok_all['City'][indeks] = forstok_all['City'][indeks].str.split(',', n = 1,expand = True)[0]

    indeks = forstok_all[forstok_all['Kecamatan'].astype(str).str.contains(',')]['Kecamatan'].index.to_list()
    if len(indeks)>0:
        forstok_all['Kelurahan'][indeks] = forstok_all['Kecamatan'][indeks].str.split(',', n = 1,expand = True)[1]
        forstok_all['Kecamatan'][indeks] = forstok_all['Kecamatan'][indeks].str.split(',', n = 1,expand = True)[0]

    forstok_all['City'] = forstok_all['City'].astype(str).str.replace('Kab\.', 'Kabupaten' ,case = False)

    master_map = pd.read_csv(r'All Data/Province.csv', names = ['Kode Prov', 'Province'], header= 0)
    master_map2 = pd.read_csv(r'All Data/City.csv', names = ['Kode City', 'Kode Prov', 'City'], header = 0)
    master_map = master_map.merge(master_map2, how = 'right', on = 'Kode Prov')
    master_map['Kode Prov'][515] = 14
    master_map['Province'][515] = 'Riau'
    master_map['Kode Prov'] = master_map['Kode Prov'].astype(int)
    master_map['Province'] = master_map['Province'].str.title()
    master_map['City'] = master_map['City'].str.title()

    city = pd.read_excel(r'All Data/list_city.xlsx')
    temp = forstok_all.copy()
    temp['City'] = temp['City'].astype(str).str.lower()
    temp['City'] = temp['City'].astype(str).str.replace('kab. ', 'kabupaten ', regex = False, case = False)
    city['All City'] = city['All City'].astype(str).str.lower()
    temp = temp.merge(city.drop_duplicates('All City'), how = 'left', left_on = 'City', right_on = 'All City').set_index(temp.index)
    indeks = temp[temp['Real City'].notnull()].index.to_list()
    forstok_all['City'][indeks] = temp['Real City'][indeks]

    province = pd.read_excel(r'All Data/list_province.xlsx')
    temp = forstok_all.copy()
    temp['Region'] = temp['Region'].astype(str).str.lower()
    province['All Province'] = province['All Province'].astype(str).str.lower()
    temp = temp.merge(province.drop_duplicates('All Province'), how = 'left', left_on = 'Region', right_on = 'All Province').set_index(temp.index)
    indeks = temp[temp['Real Province'].notnull()].index.to_list()
    forstok_all['Region'][indeks] = temp['Real Province'][indeks]

    temp = forstok_all.copy()
    temp = temp[temp['Region'].isnull()]
    temp['Region'] = temp.merge(master_map, how = 'left', on = 'City').set_index(temp.index)['Province']
    forstok_all['Region'][temp.index] = temp['Region']  
    
    district = pd.read_excel(r'All Data/list_district.xlsx')
    temp = forstok_all.copy()
    temp['Kecamatan'] = temp['Kecamatan'].astype(str).str.lower()
    district['All District'] = district['All District'].astype(str).str.lower()
    temp = temp.merge(district.drop_duplicates('All District'), how = 'left', left_on = 'Kecamatan', right_on = 'All District').set_index(temp.index)
    indeks = temp[temp['Real District'].notnull()].index.to_list()
    forstok_all['Kecamatan'][indeks] = temp['Real District'][indeks]

    temp = forstok_all.copy()
    temp2 = temp[['Region', 'City', 'Kecamatan']].merge(master_map, how = 'left', on = 'City')
    indeks = temp2[temp2['Region'] != temp2['Province']][temp2[temp2['Region'] != temp2['Province']]['City'].notnull()].index.to_list()
    forstok_all['City'][indeks] = np.nan

    data_SKU['Real SKU'] = data_SKU['SKU'].astype(str)
    data_SKU['Real Nama Produk'] = data_SKU['Nama Produk'].astype(str)

    print("Unbundling ===== 9/10")
    data_bundle1 = forstok_all[~forstok_all['Produk 1'].isnull()]
    data_bundle1['Bundle Name'] = data_bundle1['Product Name']
    data_bundle1['Bundle SKU'] = data_bundle1['SKU']
    data_bundle1['Product Name'] = data_bundle1['Produk 1']
    data_bundle1['SKU'] = data_bundle1['SKU Produk 1']
    data_bundle1['Qty. Invoiced'] = data_bundle1['PCS Produk 1']
    data_bundle1['Price List NFI'] = data_bundle1['Price List NFI 1']
    data_bundle1['Total Net'] = data_bundle1['Price List NFI 1']
    data_bundle1['Bundle Flag'] = np.nan

    data_bundle2 = forstok_all[~forstok_all['Produk 2'].isnull()]
    data_bundle2['Bundle Name'] = data_bundle2['Product Name']
    data_bundle2['Bundle SKU'] = data_bundle2['SKU']
    data_bundle2['Product Name'] = data_bundle2['Produk 2']
    data_bundle2['SKU'] = data_bundle2['SKU Produk 2']
    data_bundle2['Qty. Invoiced'] = data_bundle2['PCS Produk 2']
    data_bundle2['Price List NFI'] = data_bundle2['Price List NFI 2']
    data_bundle2['Total Net'] = data_bundle2['Price List NFI 2'] 
    data_bundle2['Bundle Flag'] = np.nan

    data_bundle3 = forstok_all[~forstok_all['Produk 3'].isnull()]
    data_bundle3['Bundle Name'] = data_bundle3['Product Name']
    data_bundle3['Bundle SKU'] = data_bundle3['SKU']
    data_bundle3['Product Name'] = data_bundle3['Produk 3']
    data_bundle3['SKU'] = data_bundle3['SKU Produk 3']
    data_bundle3['Qty. Invoiced'] = data_bundle3['PCS Produk 3']
    data_bundle3['Price List NFI'] = data_bundle3['Price List NFI 3']
    data_bundle3['Total Net'] = data_bundle3['Price List NFI 3'] 
    data_bundle3['Bundle Flag'] = np.nan

    data_bundle4 = forstok_all[~forstok_all['Produk 4'].isnull()]
    data_bundle4['Bundle Name'] = data_bundle4['Product Name']
    data_bundle4['Bundle SKU'] = data_bundle4['SKU']
    data_bundle4['Product Name'] = data_bundle4['Produk 4']
    data_bundle4['SKU'] = data_bundle4['SKU Produk 4']
    data_bundle4['Qty. Invoiced'] = data_bundle4['PCS Produk 4']
    data_bundle4['Price List NFI'] = data_bundle4['Price List NFI 4']
    data_bundle4['Total Net'] = data_bundle4['Price List NFI 4'] 
    data_bundle4['Bundle Flag'] = np.nan

    data_bundle5 = forstok_all[~forstok_all['Produk 5'].isnull()]
    data_bundle5['Bundle Name'] = data_bundle5['Product Name']
    data_bundle5['Bundle SKU'] = data_bundle5['SKU']
    data_bundle5['Product Name'] = data_bundle5['Produk 5']
    data_bundle5['SKU'] = data_bundle5['SKU Produk 5']
    data_bundle5['Qty. Invoiced'] = data_bundle5['PCS Produk 5']
    data_bundle5['Price List NFI'] = data_bundle5['Price List NFI 5']
    data_bundle5['Total Net'] = data_bundle5['Price List NFI 5']
    data_bundle5['Bundle Flag'] = np.nan

    data_bundle6 = forstok_all[~forstok_all['Produk 6'].isnull()]
    data_bundle6['Bundle Name'] = data_bundle6['Product Name']
    data_bundle6['Product Name'] = data_bundle6['Produk 6']
    data_bundle6['Bundle SKU'] = data_bundle6['SKU']
    data_bundle6['SKU'] = data_bundle6['SKU Produk 6']
    data_bundle6['Qty. Invoiced'] = data_bundle6['PCS Produk 6']
    data_bundle6['Price List NFI'] = data_bundle6['Price List NFI 6']
    data_bundle6['Total Net'] = data_bundle6['Price List NFI 6']
    data_bundle6['Bundle Flag'] = np.nan

    data_bundle7 = forstok_all[~forstok_all['Produk 7'].isnull()]
    data_bundle7['Bundle Name'] = data_bundle7['Product Name']
    data_bundle7['Bundle SKU'] = data_bundle7['SKU']
    data_bundle7['Product Name'] = data_bundle7['Produk 7']
    data_bundle7['SKU'] = data_bundle7['SKU Produk 7']
    data_bundle7['Qty. Invoiced'] = data_bundle7['PCS Produk 7']
    data_bundle7['Price List NFI'] = data_bundle7['Price List NFI 7']
    data_bundle7['Total Net'] = data_bundle7['Price List NFI 7']
    data_bundle7['Bundle Flag'] = np.nan

    data_bundle = data_bundle1._append([data_bundle2, data_bundle3, data_bundle4, data_bundle5, data_bundle6, data_bundle7], ignore_index = True, sort = False)
    data_bundle['SKU'] = data_bundle['SKU'].astype(str)
    data_bundle['SKU'] = data_bundle['SKU'].str.replace('\.0$', '', regex = True)
    data_bundle[['Real SKU', 'Real Nama Produk', 'Brand', 'Sub Brand', 'Parent Item', 'Parent SKU']] = data_bundle.merge(data_SKU[['Real SKU', 'Nama Produk', 'Brand', 'Sub Brand', 'Parent Item', 'Parent SKU']].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU')[['Real SKU_y', 'Nama Produk', 'Brand_y', 'Sub Brand_y', 'Parent Item_y', 'Parent SKU_y']]

    temp = data_bundle[data_bundle['Real SKU'].isnull()].copy()
    temp['SKU'] = temp['SKU'].astype(str).str.replace('(S)','', regex = False)
    temp = temp.merge(data_SKU[['Real SKU', 'Nama Produk', 'Brand', 'Sub Brand', 'Parent Item', 'Parent SKU']].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU').set_index(temp.index)

    indeks = data_bundle[data_bundle['Real SKU'].isnull()].index.to_list()
    data_bundle['Real SKU'][indeks] = temp['Real SKU_y'][indeks]
    data_bundle['Real Nama Produk'][indeks] = temp['Nama Produk'][indeks]
    data_bundle['Brand'][indeks] = temp['Brand_y'][indeks]
    data_bundle['Sub Brand'][indeks] = temp['Sub Brand_y'][indeks]
    data_bundle['Parent Item'][indeks] = temp['Parent Item_y'][indeks]
    data_bundle['Parent SKU'][indeks] = temp['Parent SKU_y'][indeks]

    print("Pricing ===== 10/10")
    forstok_all = forstok_all._append(data_bundle, ignore_index = True, sort = False)
    
    forstok_all = forstok_all.merge(data_SKU[['SKU', 'Price List NFI', 'Harga Cost']].drop_duplicates('SKU'), how = 'left', left_on = 'Real SKU', right_on = 'SKU').set_index(forstok_all.index)
    forstok_all['Price List NFI_x'] = forstok_all['Price List NFI_x'].fillna(forstok_all['Price List NFI_y'])
    forstok_all =  forstok_all.drop(['Price List NFI_y', 'SKU_y'], axis = 1)
    forstok_all = forstok_all.rename(columns = {'SKU_x' : 'SKU', 'Price List NFI_x' : 'Price List NFI'})

    forstok_all['Price List NFI'] = pd.to_numeric(forstok_all['Price List NFI']).astype(int)
    forstok_all['Harga Cost'] = pd.to_numeric(forstok_all['Harga Cost']).astype(int)
    forstok_all['Qty. Invoiced'] = pd.to_numeric(forstok_all['Qty. Invoiced']).astype(int)
    
    forstok_all['Total Net'] = forstok_all['Price List NFI'] * forstok_all['Qty. Invoiced']
    forstok_all['Total Harga Cost'] = forstok_all['Harga Cost'] * forstok_all['Qty. Invoiced']
    
    forstok_all = forstok_all.reset_index(drop = True)
    forstok_all['Order #'] = forstok_all['Order #'].astype(str).str.replace('.0', '', regex = False)
    before = os.listdir(os.getcwd() + '/Input Data')

#     options = Options()
#     options.add_experimental_option("prefs", {
#             "download.default_directory": os.path.abspath("Input Data"),
#             "download.directory_upgrade": True,
#             "safebrowsing_for_trusted_sources_enabled": False,
#             "safebrowsing.enabled": False
#     })

#     driver = webdriver.Chrome(options=options)
#     driver.maximize_window()
#     driver.get("https://seller.blibli.com/sign-in")
#     time.sleep(40)

#     username = driver.find_element(By.ID, "email")
#     username.clear()
#     username.send_keys("customer1@nutrimart.co.id")

#     password = driver.find_element(By.ID, "password")
#     password.send_keys("NutrimartNutrif00d")

#     driver.find_element(By.ID, "sign-in").click()
#     time.sleep(40)

#     print('login blibli')
    
#     driver.get("https://seller.blibli.com/MTA/order/summary")
#     time.sleep(15)
#     WebDriverWait(driver, 40).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[7]/div/div/div/div/div[4]/button'))).click()
#     time.sleep(15)
#     WebDriverWait(driver, 40).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[3]/div[1]/div/div/div[2]/div[2]/div[3]/button'))).click()
#     time.sleep(10)
#     driver.find_element(By.ID, "semuaFilter").click()

#     driver.find_element(By.CLASS_NAME, "order-filter-input").click()
#     start_date = datetime.today() - timedelta(days=14)
#     end_date = datetime.today()
#     date = start_date.strftime('%Y-%m-%d') + '-' + end_date.strftime('%Y-%m-%d')

#     if start_date.month == end_date.month:
#         text = "//div[@class='calendar left']//td[@class = 'available' or @class='weekend available'][text() = " + str(start_date.day) + "]"
#         driver.find_element(By.XPATH, text).click()
#         text = "//div[@class='calendar left']//td[contains(@class, 'today')][text() = " + str(end_date.day) + "]"
#         driver.find_element(By.XPATH, text).click()
#     else :
#         driver.find_element(By.CSS_SELECTOR, "th.prev.available").click()
#         text = "//div[@class='calendar left']//td[@class = 'available' or @class='weekend available'][text() = " + str(start_date.day) + "]"
#         driver.find_element(By.XPATH, text).click()
#         text = "//div[@class='calendar right']//td[contains(@class, 'today')][text() = " + str(end_date.day) + "]"
#         driver.find_element(By.XPATH, text).click()
#     driver.find_element(By.CSS_SELECTOR, 'button.btn.col-sm-3.btn-default.order-option-buttons.dropdown-toggle').click()
#     driver.find_element(By.XPATH, "/html/body/div[3]/div[2]/div/div/div[1]/div[3]/div[2]/ul").click()
#     try:
#         time.sleep(10)
#         driver.find_element(By.CLASS_NAME, "btn-primary-mta").click()
#         time.sleep(10)
#     except NoSuchElementException:
#         pass

#     print('Blibli File Download Preparation')
    
#     cond = False
#     while not cond:
#         time.sleep(30)
#         driver.find_element(By.CLASS_NAME, 'fa-bell-o').click()
#         print('click 1')
#         time.sleep(20)
#         driver.find_element(By.CLASS_NAME, 'fa-bell-o').click()
#         print('click 2')
#         time.sleep(20)
#         driver.find_element(By.CLASS_NAME, 'fa-bell-o').click()
#         print('click 3')

#         time_now = str(datetime.now().hour) + ':' + str(datetime.now().minute)
#         time.sleep(10)
#         time_notif = driver.find_element(By.XPATH, '//*[@id="viewNotification"]/div[2]/div[2]/ul/li[1]/span[2]').text
#         print(time_notif)
#         time_notif1 = driver.find_element(By.CSS_SELECTOR, 'li.ng-scope.notification-unread').text[-5:]
#         end = datetime.strptime(time_notif, '%H:%M').time()
#         print(end)
#         duration = int((datetime.today() - timedelta(hours=end.hour, minutes=end.minute)).strftime('%M'))
#         print(duration)
#         if duration < 40:
#             time.sleep(1)
#             driver.find_element(By.XPATH, '//ul[@class = "notification-item-container"]//li[@class = "ng-scope notification-unread"]').click()
#             cond = True
#         else :
#             driver.find_element(By.CLASS_NAME, 'fa-bell-o').click()
#             time.sleep(10)
#             driver.find_element(By.CLASS_NAME, 'fa-bell-o').click()
#             time.sleep(10)
            
#     print('Blibli LMS Downloaded')     

   
#     time.sleep(15)
#     after = os.listdir(os.getcwd() + '/Input Data')
#     change = set(after) - set(before)
#     if len(change) == 1:
#         file_name = change.pop()
#     else:
#         print("More than one file or no file downloaded")
#     print(file_name)
#     new_name = 'Blibli ' + str(date) + '.csv'
#     if os.path.isfile('Input Data/' + str(new_name)) :
#         os.remove('Input Data/' + str(new_name))
#     os.rename('Input Data/' + str(file_name), 'Input Data/' + str(new_name))
#     driver.quit()
#     print("Import Data ====== 1/10")
    
#     lmen = pd.read_csv(r'Input Data/' + str(new_name) , dayfirst = True)
    lmen = pd.read_csv(r"C:\Users\Steven.Nathanael\Downloads\bulk-order-download-template.csv" , dayfirst = True)
    lmen = lmen.dropna(how = 'all')
    lmen_pure = lmen.copy()

    pd.options.mode.chained_assignment = None
    print("--- %s seconds ---" % (time.time() - start_time))
    print("Formatting Data ====== 2/10")
    lmen['No. Order'] = lmen['No. Order'].astype(str).str.replace('^="','')
    lmen['No. Order'] = lmen['No. Order'].astype(str).str.replace('"$','')
    lmen['No. Order Item'] = lmen['No. Order Item'].astype(str).str.replace('^="','')
    lmen['No. Order Item'] = lmen['No. Order Item'].astype(str).str.replace('"$','')
    lmen['No. Awb'] = lmen['No. Awb'].astype(str).str.replace('^="','')
    lmen['No. Awb'] = lmen['No. Awb'].astype(str).str.replace('"$','')
    lmen['Merchant SKU'] = lmen['Merchant SKU'].astype(str).str.replace('=','')
    lmen['Merchant SKU'] = lmen['Merchant SKU'].astype(str).str.replace('"','')
    lmen['Merchant SKU'] = lmen['Merchant SKU'].astype(str).str.replace('"$','')
    
    #INPUT SKU BLIBLI LMS ERROR
    lmen.loc[lmen['Nama Produk']=='Daily Protein Package - L-Men Daily Popcorn Caramel 250g & L-Men Protein Bar Crunchy Chocolate 12 Sachet','Merchant SKU']='PL19L23'
    lmen.loc[lmen['Nama Produk']=='[Tebus Murah] L-Men Lose Weight Mango Sticky Rice 300g - Suplemen Penurun Berat Badan Tinggi Whey Protein ','Merchant SKU']='2304576112'
    lmen.loc[lmen['Nama Produk']=='WHS - FREE PROTEIN CRUNCH - L-Men PlantProtein Ogura 216g - Suplemen Tinggi Protein Nabati Rendah Lemak Vegan Lactose Free','Merchant SKU']='PL24B102'
    lmen.loc[lmen['Nama Produk']=='WHS - L-Men Platinum Ketan Hitam 800g (25g protein) - Suplemen Tinggi Whey Protein Rendah Lemak','Merchant SKU']='2305545288'
    lmen.loc[lmen['Nama Produk']=='WHS- Paket High Protein Snack - L-Men Protein Bar & 2 pcs L-Men Protein Crunch','Merchant SKU']='PL18(2)L19'
    lmen.loc[lmen['Nama Produk']=='WHS - L-Men Protein Crunch BBQ Beef (20gr)','Merchant SKU']='2309005305'
    lmen.loc[lmen['Nama Produk'].isin(['FREE SPIDER BOTTLE ']),'Merchant SKU']='(B)71210026'
    lmen.loc[lmen['Blibli SKU'].isin(['LMS-60022-00307-00001']),'Merchant SKU']='(G)71310033'
    lmen.loc[lmen['Blibli SKU'].isin(['LMS-60022-00306-00001']),'Merchant SKU']='(G)71310032'
    lmen.loc[lmen['Blibli SKU'].isin(['LMS-60022-00311-00001']),'Merchant SKU']='2307061250P4'
    lmen.loc[lmen['Blibli SKU'].isin(['LMS-60022-00310-00001']),'Merchant SKU']='(E)2304576112P2'
    lmen.loc[lmen['Blibli SKU'].isin(['LMS-60022-00309-00001']),'Merchant SKU']='(E)2304576112'
    lmen.loc[lmen['Blibli SKU'].isin(['LMS-60022-00314-00001']),'Merchant SKU']='PL7(2)B102(2)'
    lmen.loc[lmen['Blibli SKU'].isin(['LMS-60022-00313-00001']),'Merchant SKU']='2304515112P2'
    lmen.loc[lmen['Blibli SKU'].isin(['LMS-60022-00319-00001']),'Merchant SKU']='(E)2305551288'
    lmen.loc[lmen['Blibli SKU'].isin(['LMS-60022-00318-00001']),'Merchant SKU']='(E)2305551288P2'
    lmen.loc[lmen['Blibli SKU'].isin(['LMS-60022-00322-00001']),'Merchant SKU']='(E)2LL1402249P24'
    lmen.loc[lmen['Blibli SKU'].isin(['LMS-60022-00323-00001']),'Merchant SKU']='PL3G128'
    lmen.loc[lmen['Blibli SKU'].isin(['LMS-60022-00324-00001']),'Merchant SKU']='PE66E80'
    lmen.loc[lmen['Blibli SKU'].isin(['LMS-60022-00325-00001']),'Merchant SKU']='(E)2305559288'
    lmen.loc[lmen['Blibli SKU'].isin(['LMS-60022-00320-00001']),'Merchant SKU']='(E)2LL1402249P4'
    lmen.loc[lmen['Blibli SKU'].isin(['LMS-60022-00330-00001']),'Merchant SKU']='(E)2305584288'
    lmen.loc[lmen['Blibli SKU'].isin(['LMS-60022-00332-00001']),'Merchant SKU']='(E)2305551288P3'
    lmen.loc[lmen['Blibli SKU'].isin(['LMS-60022-00333-00001']),'Merchant SKU']='(E)2301050106'
    lmen.loc[lmen['Blibli SKU'].isin(['LMS-60022-00335-00001']),'Merchant SKU']='(E)2304036151'
     
    for i in lmen[lmen['Merchant SKU'].isnull()]['Blibli SKU'].unique():
        print(i)
        skunfi=lmen[(lmen['Blibli SKU'] == i)&(lmen['Merchant SKU'].isnull())].reset_index()['Merchant SKU'][0]
        lmen.loc[(lmen['Blibli SKU']==i)&(lmen['Merchant SKU'].isnull()),'Merchant SKU']=skunfi

    print("--- %s seconds ---" % (time.time() - start_time))
    print("Listing SKU ====== 3/10")
    lmen_sku = pd.read_excel(r'SKU_File/Converter Paket Blibli.xlsx')

    lmen['Merchant SKU'] = lmen['Merchant SKU'].astype(str).str.replace('\.0$','', regex = True)

    null_sku = lmen[lmen['Merchant SKU'].isnull()].copy()
    lmen = lmen[~lmen['Merchant SKU'].isnull()]

    null_sku['Nama Produk'] = null_sku['Nama Produk'].astype(str).str.strip()
    lmen_sku['Nama Produk'] = lmen_sku['Nama Produk'].astype(str).str.strip()
    null_sku['Merchant SKU'] = null_sku.merge(lmen_sku[['SKU Merchant', 'Nama Produk']].drop_duplicates(['Nama Produk']), how = 'left', on = 'Nama Produk').set_index(null_sku.index)['SKU Merchant']

    lmen = lmen._append(null_sku, ignore_index = True, sort = False)

    gaada_sku = lmen[~lmen['Merchant SKU'].astype(str).isin(data_SKU['SKU'].astype(str))].copy()
    lmen = lmen[lmen['Merchant SKU'].astype(str).isin(data_SKU['SKU'].astype(str))]

    gaada_sku['Nama Produk'] = gaada_sku['Nama Produk'].astype(str).str.strip()
    lmen_sku['Nama Produk'] = lmen_sku['Nama Produk'].astype(str).str.strip()
    gaada_sku['Merchant SKU'] = gaada_sku.merge(lmen_sku[['SKU Merchant', 'Nama Produk']].drop_duplicates(['Nama Produk']), how = 'left', on = 'Nama Produk').set_index(gaada_sku.index)['SKU Merchant']
    lmen = lmen._append(gaada_sku, ignore_index = True, sort = False)

    indeks = []
    indeks = indeks + lmen[lmen['Merchant SKU'].isnull()].index.to_list()
    indeks = indeks + lmen[~lmen['Merchant SKU'].astype(str).str.replace('\.0$','', regex = True).isin(data_SKU['SKU'].astype(str))].index.to_list()

    if len(indeks) != 0 :
        alert = lmen.iloc[indeks][['Blibli SKU','Merchant SKU', 'Nama Produk']].drop_duplicates()
        alert['SKU Valid'] = np.nan
        to_excel = alert.to_excel('ALERT_LMEN-STORE_SKU_MISSING.xlsx')
        print("Some SKU Missing Please Complete It ====== 4/10")
        print("--- %s seconds ---" % (time.time() - start_time))
    else :
        data_SKU['Real Nama Produk'] = data_SKU['Nama Produk'].astype(str)
        data_SKU['Real SKU'] = data_SKU['SKU'].astype(str)
        lmen['Merchant SKU'] = lmen['Merchant SKU'].astype(str)
        data_SKU.loc[data_SKU['Price List NFI'].isin(['-']),'Price List NFI'] = 0
        data_SKU['Price List NFI'] = data_SKU['Price List NFI'].astype(float).astype('int64')
        lmen = lmen.merge(data_SKU[['Real SKU', 'Real Nama Produk', 'Brand', 'Sub Brand', 'Parent Item', 'Parent SKU', 'Price List NFI']].drop_duplicates(['Real SKU']), how = 'left', left_on = 'Merchant SKU', right_on = 'Real SKU')
        lmen['Total Net'] = lmen['Price List NFI'] * lmen['Total Barang']

        indeks = lmen[lmen['Brand'] == 'Bundle'].index.to_list()
        lmen['Bundle Flag'] = np.nan
        lmen['Bundle Flag'][indeks] = 'Bundle'

        lmen['Date'] = pd.to_datetime(lmen['Tanggal Order'], format = '%d/%m/%Y %H:%M').dt.day
        lmen['Month'] = pd.to_datetime(lmen['Tanggal Order'], format = '%d/%m/%Y %H:%M').dt.month_name()
        lmen['Year'] = pd.to_datetime(lmen['Tanggal Order'], format = '%d/%m/%Y %H:%M').dt.year
        lmen['Hour'] = pd.to_datetime(lmen['Tanggal Order'], format = '%d/%m/%Y %H:%M').dt.hour
        lmen['Minute'] = pd.to_datetime(lmen['Tanggal Order'], format = '%d/%m/%Y %H:%M').dt.minute
        lmen['Second'] = pd.to_datetime(lmen['Tanggal Order'], format = '%d/%m/%Y %H:%M').dt.second
        

        quarter = pd.DataFrame([['January', 1], ['February', 1], ['March', 1], ['April', 2], ['May', 2], ['June', 2], 
                    ['July', 3], ['August', 3], ['September', 3],['October', 4], ['November', 4], ['December', 4]], columns = ['Bulan', 'Quarter'])
        lmen = lmen.merge(quarter, how = 'left', left_on = 'Month', right_on = 'Bulan')
        lmen = lmen.drop(['Bulan'], axis = 1)
        data_bulan = pd.DataFrame([{'Bulan' : 'December', 'Number' : 12} ,
                {'Bulan' : 'January' , 'Number': 1},
                {'Bulan' : 'February' , 'Number': 2},
                {'Bulan' : 'March' , 'Number': 3},
                {'Bulan' : 'April' , 'Number': 4},
                {'Bulan' : 'May' , 'Number': 5},
                {'Bulan' : 'June', 'Number': 6},
                {'Bulan' : 'July' , 'Number': 7},
                {'Bulan' : 'August', 'Number' : 8},
                {'Bulan' : 'September', 'Number' : 9},
                {'Bulan' : 'October' , 'Number': 10},
                {'Bulan' : 'November' , 'Number': 11}])
        temp = lmen.copy()
        temp['Day'] = temp['Date']
        temp = temp.merge(data_bulan, how = 'left', left_on = 'Month', right_on='Bulan')
        temp= temp.rename(columns = {'Month' : 'Bulan', 'Number' : 'Month'})
        lmen['Week'] = pd.to_datetime(temp[['Year', 'Month', 'Day']]).dt.strftime("%U")
        lmen['True datetime'] = pd.to_datetime(temp[['Year', 'Month', 'Day', 'Hour', 'Minute', 'Second']])

        lmen = lmen.rename(columns = {'No. Order' : 'Order #', 'No. Order Item' : 'No. Order Item L-Men Blibli',
                                    'Tanggal Order' : 'Order date', 'Nama Pemesan' : 'Customer Name',
                                    'Merchant SKU' : 'SKU', 'Nama Produk' : 'Product Name', 'Total Barang':'Qty. Invoiced',
                                    'Servis Logistik' : 'Shipping Courier', 'Nama Store' : 'Channel',
                                    'Toko/Gudang' : 'Warehouse Name', 'No. Awb' : 'AWB'
                                    })

        lmen['Selling Price'] = (lmen['Qty. Invoiced']-lmen['Diskon'])/lmen['Qty. Invoiced']
        lmen['Channel'] = 'L-Men Store Blibli'
        lmen['Store'] = 'Blibli'

        data_SKU['Real SKU'] = data_SKU['SKU'].astype(str)
        data_SKU['Real Nama Produk'] = data_SKU['Nama Produk'].astype(str)

        lmen['SKU'] = lmen['SKU'].astype(str)
        list_col = ['SKU'] + data_SKU.columns[data_SKU.columns.get_loc('Produk 1'):data_SKU.columns.get_loc('Harga Cost 7')+1].to_list()
        lmen = lmen.merge(data_SKU[list_col].drop_duplicates(['SKU']), how = 'left', left_on = 'Real SKU', right_on = 'SKU')
        lmen = lmen.drop(['SKU_y'], axis = 1)
        lmen = lmen.rename(columns = {'SKU_x':'SKU'})
        
        data_bundle1 = lmen[~lmen['Produk 1'].isnull()]
        data_bundle1['Bundle Name'] = data_bundle1['Product Name']
        data_bundle1['Bundle SKU'] = data_bundle1['SKU']
        data_bundle1['Product Name'] = data_bundle1['Produk 1']
        data_bundle1['SKU'] = data_bundle1['SKU Produk 1']
        data_bundle1['Qty. Invoiced'] = data_bundle1['PCS Produk 1'] * data_bundle1['Qty. Invoiced']
        data_bundle1['Price List NFI'] = data_bundle1['Price List NFI 1']
        data_bundle1['Total Net'] = data_bundle1['Price List NFI 1'] * data_bundle1['Qty. Invoiced']
        data_bundle1['Bundle Flag'] = np.nan

        data_bundle2 = lmen[~lmen['Produk 2'].isnull()]
        data_bundle2['Bundle Name'] = data_bundle2['Product Name']
        data_bundle2['Bundle SKU'] = data_bundle2['SKU']
        data_bundle2['Product Name'] = data_bundle2['Produk 2']
        data_bundle2['SKU'] = data_bundle2['SKU Produk 2']
        data_bundle2['Qty. Invoiced'] = data_bundle2['PCS Produk 2'] * data_bundle2['Qty. Invoiced']
        data_bundle2['Price List NFI'] = data_bundle2['Price List NFI 2']
        data_bundle2['Total Net'] = data_bundle2['Price List NFI 2'] * data_bundle2['Qty. Invoiced']
        data_bundle2['Bundle Flag'] = np.nan

        data_bundle3 = lmen[~lmen['Produk 3'].isnull()]
        data_bundle3['Bundle Name'] = data_bundle3['Product Name']
        data_bundle3['Bundle SKU'] = data_bundle3['SKU']
        data_bundle3['Product Name'] = data_bundle3['Produk 3']
        data_bundle3['SKU'] = data_bundle3['SKU Produk 3']
        data_bundle3['Qty. Invoiced'] = data_bundle3['PCS Produk 3']* data_bundle3['Qty. Invoiced']
        data_bundle3['Price List NFI'] = data_bundle3['Price List NFI 3']
        data_bundle3['Total Net'] = data_bundle3['Price List NFI 3'] * data_bundle3['Qty. Invoiced']
        data_bundle3['Bundle Flag'] = np.nan

        data_bundle4 = lmen[~lmen['Produk 4'].isnull()]
        data_bundle4['Bundle Name'] = data_bundle4['Product Name']
        data_bundle4['Bundle SKU'] = data_bundle4['SKU']
        data_bundle4['Product Name'] = data_bundle4['Produk 4']
        data_bundle4['SKU'] = data_bundle4['SKU Produk 4']
        data_bundle4['Qty. Invoiced'] = data_bundle4['PCS Produk 4']* data_bundle4['Qty. Invoiced']
        data_bundle4['Price List NFI'] = data_bundle4['Price List NFI 4']
        data_bundle4['Total Net'] = data_bundle4['Price List NFI 4'] * data_bundle4['Qty. Invoiced']
        data_bundle4['Bundle Flag'] = np.nan

        data_bundle5 = lmen[~lmen['Produk 5'].isnull()]
        data_bundle5['Bundle Name'] = data_bundle5['Product Name']
        data_bundle5['Bundle SKU'] = data_bundle5['SKU']
        data_bundle5['Product Name'] = data_bundle5['Produk 5']
        data_bundle5['SKU'] = data_bundle5['SKU Produk 5']
        data_bundle5['Qty. Invoiced'] = data_bundle5['PCS Produk 5']* data_bundle5['Qty. Invoiced']
        data_bundle5['Price List NFI'] = data_bundle5['Price List NFI 5']
        data_bundle5['Total Net'] = data_bundle5['Price List NFI 5'] * data_bundle5['Qty. Invoiced']
        data_bundle5['Bundle Flag'] = np.nan

        data_bundle6 = lmen[~lmen['Produk 6'].isnull()]
        data_bundle6['Bundle Name'] = data_bundle6['Product Name']
        data_bundle6['Bundle SKU'] = data_bundle6['SKU']
        data_bundle6['Product Name'] = data_bundle6['Produk 6']
        data_bundle6['SKU'] = data_bundle6['SKU Produk 6']
        data_bundle6['Qty. Invoiced'] = data_bundle6['PCS Produk 6']* data_bundle6['Qty. Invoiced']
        data_bundle6['Price List NFI'] = data_bundle6['Price List NFI 6']
        data_bundle6['Total Net'] = data_bundle6['Price List NFI 6'] * data_bundle6['Qty. Invoiced']
        data_bundle6['Bundle Flag'] = np.nan

        data_bundle7 = lmen[~lmen['Produk 7'].isnull()]
        data_bundle7['Bundle Name'] = data_bundle7['Product Name']
        data_bundle7['Bundle SKU'] = data_bundle7['SKU']
        data_bundle7['Product Name'] = data_bundle7['Produk 7']
        data_bundle7['SKU'] = data_bundle7['SKU Produk 7']
        data_bundle7['Qty. Invoiced'] = data_bundle7['PCS Produk 7']* data_bundle7['Qty. Invoiced']
        data_bundle7['Price List NFI'] = data_bundle7['Price List NFI 7']
        data_bundle7['Total Net'] = data_bundle7['Price List NFI 7'] * data_bundle7['Qty. Invoiced']
        data_bundle7['Bundle Flag'] = np.nan

        data_bundle = data_bundle1._append([data_bundle2, data_bundle3, data_bundle4, data_bundle5, data_bundle6, data_bundle7], ignore_index = True, sort = False)
        data_bundle['SKU'] = data_bundle['SKU'].astype(str)
        data_bundle['SKU'] = data_bundle['SKU'].str.replace('\.0$', '', regex = True)
        data_bundle[['Real SKU', 'Real Nama Produk', 'Brand', 'Sub Brand', 'Parent Item', 'Parent SKU']] = data_bundle.merge(data_SKU[['Real SKU', 'Nama Produk', 'Brand', 'Sub Brand', 'Parent Item', 'Parent SKU']].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU')[['Real SKU_y', 'Nama Produk', 'Brand_y', 'Sub Brand_y', 'Parent Item_y', 'Parent SKU_y']]

        lmen = lmen._append(data_bundle, ignore_index = True, sort = False)
        lmen = lmen.merge(data_SKU[['Real SKU', 'Harga Cost']].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU')
        lmen['Total Harga Cost'] = pd.to_numeric(lmen['Harga Cost'], errors = 'coerce') * lmen['Qty. Invoiced']
        print(lmen[lmen['Harga Cost'] == 0][['SKU','Product Name']].drop_duplicates())
        lmen['Total'] = lmen['Selling Price'] * lmen['Qty. Invoiced']
        lmen['Subtotal'] = lmen['Selling Price'] * lmen['Qty. Invoiced']
        lmen = lmen.rename(columns = {'Real SKU_x' : 'Real SKU'})

        forstok_all = forstok_all._append(lmen, ignore_index = True, sort = False)

        list_bundle = forstok_all[forstok_all['Bundle Flag'] == 'Bundle'][['Order #', 'Product Name', 'Subtotal', 'Total']].groupby(['Order #', 'Product Name']).sum().reset_index()
        list_nobundle = forstok_all[forstok_all['Bundle Name'].notnull()]
        list_nobundle = list_nobundle.merge(list_bundle, how = 'left', left_on = ['Order #', 'Bundle Name'], right_on = ['Order #', 'Product Name']).set_index(list_nobundle.index)
        list_nobundle

        forstok_all['Total'][list_nobundle.index] = list_nobundle['Total_y']
        forstok_all['Subtotal'][list_nobundle.index] = list_nobundle['Subtotal_y']

        temp = forstok_all[forstok_all['Bundle Name'].notnull()]
        temp['Subtotal'] = temp['Subtotal'] * temp['Total Harga Cost']/temp.groupby(['Order #', 'Bundle Name'])['Total Harga Cost'].transform('sum')
        temp['Selling Price'] = temp['Subtotal']/temp['Qty. Invoiced']
        temp['Total'] = temp['Total'] * temp['Total Harga Cost']/temp.groupby(['Order #', 'Bundle Name'])['Total Harga Cost'].transform('sum')

        forstok_all['Total'][temp.index] = temp['Total']
        forstok_all['Subtotal'][temp.index] = temp['Subtotal']
        forstok_all['Selling Price'][temp.index] = temp['Selling Price']
        

        forstok_all['Order #'] = forstok_all['Order #'].astype(str).str.replace('.0', '', regex = False)

        list_bundle = forstok_all[forstok_all['Bundle Flag'] == 'Bundle'][['Order #', 'Product Name', 'Seller Discount']].groupby(['Order #', 'Product Name']).sum().reset_index()
        list_nobundle = forstok_all[forstok_all['Bundle Name'].notnull()]
        list_nobundle = list_nobundle.merge(list_bundle, how = 'left', left_on = ['Order #', 'Bundle Name'], right_on = ['Order #', 'Product Name']).set_index(list_nobundle.index)
        list_nobundle

        forstok_all['Seller Discount'][list_nobundle.index] = list_nobundle['Seller Discount_y']
        temp = forstok_all[forstok_all['Bundle Name'].notnull()]
        temp['Seller Discount'] = temp['Seller Discount'] * temp['Total Harga Cost']/temp.groupby(['Order #', 'Bundle Name'])['Total Harga Cost'].transform('sum')
        forstok_all['Seller Discount'][temp.index] = temp['Seller Discount']

        temp = forstok_all[forstok_all['Bundle Name'].isnull()]
        temp_group = temp[['Order #','Shipping']].groupby(['Order #']).sum().reset_index()

        temp = forstok_all.merge(temp_group, how = 'left', on = 'Order #').set_index(forstok_all.index)
        temp['Shipping_x'] = temp['Shipping_y'] * temp['Total Harga Cost']/temp.groupby(['Order #', 'Bundle Name'])['Total Harga Cost'].transform('sum')

        forstok_all['Shipping'][temp.index] = temp['Shipping_y']
        list_bundle = forstok_all[forstok_all['Bundle Flag'] == 'Bundle'][['Order #', 'Product Name', 'Shipping']].groupby(['Order #', 'Product Name']).sum().reset_index()
        list_nobundle = forstok_all[forstok_all['Bundle Name'].notnull()]
        list_nobundle = list_nobundle.merge(list_bundle, how = 'left', left_on = ['Order #', 'Bundle Name'], right_on = ['Order #', 'Product Name']).set_index(list_nobundle.index)
        list_nobundle

        forstok_all['Shipping'][list_nobundle.index] = list_nobundle['Shipping_y']
        temp = forstok_all[forstok_all['Bundle Name'].notnull()]
        temp['Shipping'] = temp['Shipping'] * temp['Total Harga Cost']/temp.groupby(['Order #', 'Bundle Name'])['Total Harga Cost'].transform('sum')
        forstok_all['Shipping'][temp.index] = temp['Shipping']
        forstok_all['True datetime'] = pd.to_datetime(forstok_all['True datetime'])
        forstok_all['Promo'] = np.nan
        forstok_all['Discount MC'] = np.nan

print('DONE')

Input Forstok Date
2024-03-04-2024-03-14
Opening Chrome
Login Forstok
input email
input password
click
Forstok Downloaded
Download Data SKU
Waiting to Download Forstok
https://forstok-staging-storage.s3.ap-southeast-1.amazonaws.com/items_import/Nutrifood-forstok-sales_orders-version_2-2024-03-03T17:00:00Z-2024-03-14T16:59:59Z.xlsx
Waiting to Download Forstok
Waiting to Download Forstok
Waiting to Download Forstok
Waiting to Download Forstok
Waiting to Download Forstok
Waiting to Download Forstok
Waiting to Download Forstok
Waiting to Download Forstok
Waiting to Download Forstok
Waiting to Download Forstok
Waiting to Download Forstok
Waiting to Download Forstok


In [13]:
osf=pd.read_excel(r"data_supp\order filter.xlsx")
forstok_all[forstok_all['Brand'].isin(['NS','TS','HiLo','L-Men','WDANK']) &
            forstok_all['Order Status'].isin(osf['order filter'].unique())].groupby(['Month','Date']).agg({'Total' : 'sum'})

Total
Month     Date              
April     2.0   1.467629e+09
August    2.0   6.046165e+08
December  2.0   3.376123e+08
February  1.0   7.000000e+00
          2.0   4.140500e+04
          3.0   1.630700e+04
          4.0   1.072110e+05
          5.0   1.270100e+04
          6.0   2.660400e+04
          7.0   2.900400e+04
          8.0   1.531700e+04
          9.0   2.000000e+00
          10.0  3.000000e+00
          11.0  1.200000e+01
          12.0  4.820600e+04
          13.0  4.350128e+07
July      2.0   7.178651e+08
June      2.0   1.065028e+09
March     2.0   1.474082e+09
May       2.0   1.435213e+09
November  2.0   2.995676e+08
October   2.0   2.721116e+08
September 2.0   4.848690e+08

In [ ]:
#OPTIONAL JIKA CODING ATAS ERROR

In [4]:
#CEK SKU FORSTOK YANG BELUM MASUK TATANAMA
### to do: regist single/bundle/alias 

data_forstok[data_forstok['SKU'].isnull()]['Item Name'] .unique()
data_forstok[~data_forstok['SKU'].astype(str).isin(data_SKU['SKU'].astype(str))][['Store','Item Name','SKU']].drop_duplicates().values#.unique()

array([['Tokopedia',
        'HiLo School 500gr All Variant Susu Tinggi Kalsium Rendah Lemak Honey',
        '2101401180(XXIdy)']], dtype=object)

In [ ]:
#CEK SKU L-MEN BLIBLI YANG KOSONG

lmen[lmen['Merchant SKU'].isnull()]['Blibli SKU']

In [86]:
#Pre Run-4
#Belum scraping magento dan blibli
magento_scrape = False
magentoUser_scrape = False

In [ ]:
## Run-4
## scrap magento detail order blibli update order status
## kadang auto logout di blibli -> solusi: run partial / add code

from datetime import datetime
from selenium import webdriver
from selenium.webdriver.chrome.service import Service

if not magento_scrape: 
    print("Starting Magento")

    options = Options()
    options.add_experimental_option("prefs", {
            "download.default_directory": os.path.abspath(r"C:\Users\Steven.Nathanael\Documents\2. Master Data Git\Input Data"),
            "download.directory_upgrade": True,
            "safebrowsing_for_trusted_sources_enabled": False,
            "safebrowsing.enabled": False
    })

    print('Opening Chrome')
    driver = webdriver.Chrome(options = options)
    driver.get("https://nutrimart.co.id/backoffice")
    print('Login Magento')
    username = driver.find_element(By.ID, "username")
    username.clear()
    username.send_keys("timotius")
    password = driver.find_element(By.ID, "login")
    password.send_keys("timo123")

    print('Go To Sales Form')
    time.sleep(5)
    before = os.listdir(os.getcwd() + '/Input Data')
    driver.find_element(By.XPATH, "//button[@class = 'action-login action-primary']").click()
    time.sleep(5)
    WebDriverWait(driver, 300).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="menu-magento-sales-sales"]'))).click()
    time.sleep(5)
    driver.find_element(By.XPATH, '//*[@id="menu-magento-sales-sales"]/div/ul/li[2]/ul/li[2]/div/ul/li[1]/a').click()
    driver.find_element(By.XPATH, '//*[@id="profile_id"]').click()
    driver.find_element(By.XPATH, '//*[@id="profile_id"]/optgroup[2]/option[2]').click()
    

    start_date = datetime.today() - timedelta(days=12)
    print(start_date)
    end_date = datetime.today()
    print(end_date)
    date = start_date.strftime('%d %b %Y') + '-' + end_date.strftime('%d %b %Y')

    print('Input Date')
    date_from = driver.find_element(By.NAME, 'daterange_from')
    date_from.clear()
    date_from.send_keys(start_date.strftime('%m/%d/%Y'))

    date_to = driver.find_element(By.NAME, 'daterange_to')
    date_to.clear()
    date_to.send_keys(end_date.strftime('%m/%d/%Y'))
    print('Download Magento')
    driver.find_element(By.XPATH, '//*[@id="export_button"]').click()

    time.sleep(120)
    after = os.listdir(os.getcwd() + '/Input Data')
    change = set(after) - set(before)
    if len(change) == 1:
        magento_name = change.pop()
    else:
        print("More than one file or no file downloaded")
    ###MANUAL IF MAGENTO ERROR
    #data_magento = pd.read_csv('Input Data\export_sales_detailed_report_320667.csv', sep = ',', index_col = False)
    data_magento = pd.read_csv('Input Data/' + magento_name, sep = ',', index_col = False)
    
    magento_scrape = True
else :
    ###MANUAL IF MAGENTO ERROR
    #data_magento = pd.read_csv('Input Data\export_sales_detailed_report_321223.csv', sep = ',', index_col = False)
    
    data_magento = pd.read_csv('Input Data/' + magento_name, sep = ',', index_col = False)
    
if not magentoUser_scrape:
    before = os.listdir(os.getcwd() + '/Input Data')

    driver.find_element(By.XPATH, '//*[@id="profile_id"]').click()
    driver.find_element(By.XPATH, '//*[@id="profile_id"]/optgroup[2]/option[1]').click()

    start_date = datetime.today() - timedelta(days=14)
    end_date = datetime.today()
    date = start_date.strftime('%d %b %Y') + '-' + end_date.strftime('%d %b %Y')

    date_from = driver.find_element(By.NAME, 'daterange_from')
    date_from.clear()
    date_from.send_keys(start_date.strftime('%m/%d/%Y'))
    date_to = driver.find_element(By.NAME, 'daterange_to')
    date_to.clear()
    date_to.send_keys(end_date.strftime('%m/%d/%Y'))

    driver.find_element(By.XPATH, '//*[@id="export_button"]').click()
    time.sleep(60)
    after = os.listdir(os.getcwd() + '/Input Data')
    change = set(after) - set(before)
    if len(change) == 1:
        magentouser_name = change.pop()
        print(file_name)
    else:
        print("More than one file or no file downloaded")
        print(change)
    driver.quit()

    print(file_name)
    ###MANUAL IF MAGENTO ERROR
    #data_magentoUser2 = pd.read_csv(r"C:\Users\steven.nathanael\Documents\Python\Data\2. Master Data Git\Input Data\export_320691.csv", sep = ',', index_col = False)
    
    data_magentoUser2 = pd.read_csv('Input Data/' + magentouser_name, sep = ',', index_col = False)
    for i in range(data_magentoUser2.shape[1]):
        data_magentoUser2 = data_magentoUser2.rename(columns={ data_magentoUser2.columns[i] : data_magentoUser2.columns[i].replace("    "," ")})
    magentoUser_scrape = True
else :
    ###MANUAL IF MAGENTO ERROR
    #data_magentoUser2 = pd.read_csv(r"C:\Users\steven.nathanael\Documents\Python\Data\2. Master Data Git\Input Data\export_321224.csv", sep = ',', index_col = False)
    
    data_magentoUser2 = pd.read_csv('Input Data/' + magentouser_name, sep = ',', index_col = False)
    for i in range(data_magentoUser2.shape[1]):
        data_magentoUser2 = data_magentoUser2.rename(columns={ data_magentoUser2.columns[i] : data_magentoUser2.columns[i].replace("    "," ")})

data_SKU = pd.read_excel(r"SKU_File\data_SKU.xlsx")

s = requests.Session()
s.get("https://tatanama.pythonanywhere.com")
s.post("https://tatanama.pythonanywhere.com", data = {'username' : 'ecommerce', 'password' : 'ecommerce'})
r = s.get("https://tatanama.pythonanywhere.com/download")

with open(r"SKU_File\Master tatanama.xlsx", 'wb') as output:
    output.write(r.content)

if os.path.isfile(r"SKU_File\Master tatanama.xlsx") :    
    SKU_append = pd.read_excel(r"SKU_File\Master tatanama.xlsx")
    SKU_append.columns = [x.replace('_', ' ') for x in SKU_append.columns]
    data_SKU = data_SKU[~data_SKU['SKU'].astype(str).isin(SKU_append['SKU'].astype(str))]
    data_SKU = data_SKU._append(SKU_append, ignore_index = True, sort = False)

to_excel = data_SKU.to_excel(r"SKU_File\data_SKU.xlsx", index = False)


# Import library
import time
import pandas as pd
import numpy as np
import datetime
import math
import os

# Import data
start_time = time.time()
print("Import Data ====== 1/10")

data_magentoUser = pd.read_excel('All Data\data_magentoUser_2020.xlsx', index_col = False)
data_magentoUser = data_magentoUser[~data_magentoUser['Sales Order Id'].astype(str).isin(data_magentoUser2['Sales Order Id'].astype(str))]
data_magentoUser = data_magentoUser._append(data_magentoUser2, ignore_index = True, sort = False)
magento_pure = data_magento.copy()

list_skumiss = []

# Formatting Magento
print("--- %s seconds ---" % (time.time() - start_time))
print("Formatting Data ====== 2/10")

pd.options.mode.chained_assignment = None
if set(['Qty. Ordered"', 'Qty. Shipped"']).issubset(data_magento.columns):
    data_magento = data_magento.rename(columns={'Qty. Ordered"' : 'Qty. Ordered', 
                                            'Qty. Shipped"' : 'Qty. Shipped'})
if "Manufacturer" in data_magento.columns:
    data_magento= data_magento.drop(["Manufacturer"], axis=1)
if "Item Cost" in data_magento.columns:
    data_magento= data_magento.drop(["Item Cost"], axis=1)
if "Package Name" in data_magento.columns:
    data_magento= data_magento.drop(["Package Name"], axis=1)
if "Tax Refunded" in data_magento.columns:
    data_magento= data_magento.drop(["Tax Refunded"], axis=1)
if "Catalog Name Rule" in data_magento.columns:
    data_magento= data_magento.drop(["Catalog Name Rule"], axis=1)
if "Order date" in data_magento.columns:
    data_magento["Order date"] = pd.to_datetime(data_magento["Order date"], errors = 'coerce')
if "Customer Email" in data_magento.columns:
    data_magento["Customer Email"] = data_magento["Customer Email"].replace(regex = r'"', value="")
if "Qty. Ordered" in data_magento.columns:
    data_magento['Qty. Ordered'] = data_magento['Qty. Ordered'].fillna(0)
    if "Qty. Invoiced" in data_magento.columns:
        data_magento['Qty. Invoiced'] = data_magento['Qty. Invoiced'].fillna(0)
    if "Qty. Shipped" in data_magento.columns:
        data_magento['Qty. Shipped'] = data_magento['Qty. Shipped'].fillna(0)
    if "Qty. Refunded" in data_magento.columns:
        data_magento['Qty. Refunded'] = data_magento['Qty. Refunded'].fillna(0)
    if "Item Price" in data_magento.columns:
        data_magento['Item Price'] = data_magento['Item Price'].fillna(0)
    if "Subtotal" in data_magento.columns:
        data_magento['Subtotal'] = data_magento['Subtotal'].fillna(0)
    if "Discounts" in data_magento.columns:
        data_magento['Discounts'] = data_magento['Discounts'].fillna(0)
    if "Invoiced" in data_magento.columns:
        data_magento['Invoiced'] = data_magento['Invoiced'].fillna(0)
    if "Tax Invoiced" in data_magento.columns:
        data_magento['Tax Invoiced'] = data_magento['Tax Invoiced'].fillna(0)
    if "Voucher Amount" in data_magento.columns:
        data_magento[data_magento['Voucher Amount'].astype(str).str.isdigit()]['Voucher Amount'] = data_magento[data_magento['Voucher Amount'].fillna(0).astype(str).str.isdigit()]['Voucher Amount'].astype(float).div(10000)
    if "Discount Poin Reward" in data_magento.columns:
        data_magento['Discount Poin Reward'] = data_magento['Discount Poin Reward'].fillna(0)
    if "Discount Product" in data_magento.columns:
        data_magento['Discount Product'] = data_magento['Discount Product'].fillna(0)
if "Tax" in data_magento.columns:
    data_magento['Tax'] = data_magento['Tax'].astype(np.float32)
if "Total" in data_magento.columns:
    data_magento['Total'] = data_magento['Total'].astype(np.float32)
if "Total incl. Tax" in data_magento.columns:
    data_magento['Total incl. Tax'] = pd.to_numeric(data_magento['Total incl. Tax'], errors = 'coerce').astype(np.float32)
if "Invoiced incl. Tax" in data_magento.columns:
    data_magento['Invoiced incl. Tax'] = data_magento['Invoiced incl. Tax'].astype(np.float32)
if "Refunded" in data_magento.columns:
    data_magento['Refunded'] = data_magento['Refunded'].astype(np.float32)
if "Refunded incl. Tax" in data_magento.columns:
    data_magento['Refunded incl. Tax'] = data_magento['Refunded incl. Tax'].astype(np.float32)
if "Ship Date" in data_magento.columns:
    data_magento['Ship Date'] = data_magento['Ship Date'].astype(str).str.replace('false', '')

# Customer Information Filling  
for i in range(data_magentoUser.shape[1]):
    data_magentoUser = data_magentoUser.rename(columns={ data_magentoUser.columns[i] : data_magentoUser.columns[i].replace("    "," ")})

for i in range(data_magento.shape[0]):
    if str(data_magento["Customer Group"][i]) == 'nan':
        if data_magento["Order #"][i] in data_magentoUser["Sales Order Id"].values:
            data_magento["Customer Name"][i] = data_magentoUser["Shipping Name"].loc[data_magento["Order #"][i]==data_magentoUser["Sales Order Id"]].values[0]
            data_magento["Customer Group"][i] = "Not Logged In"
            data_magento["Country"][i] = data_magentoUser["Shipping Country"].loc[data_magento["Order #"][i]==data_magentoUser["Sales Order Id"]].values[0]
            data_magento["Region"][i] = data_magentoUser["Shipping Province"].loc[data_magento["Order #"][i]==data_magentoUser["Sales Order Id"]].values[0]
            data_magento["City"][i] = data_magentoUser["Shipping City"].loc[data_magento["Order #"][i]==data_magentoUser["Sales Order Id"]].values[0]
            data_magento["Zip Code"][i] = data_magentoUser["Shipping Zip"].loc[data_magento["Order #"][i]==data_magentoUser["Sales Order Id"]].values[0]
            data_magento["Address"][i] = data_magentoUser["Shipping Address1"].loc[data_magento["Order #"][i]==data_magentoUser["Sales Order Id"]].values[0]
            data_magento["Phone"][i] = data_magentoUser["Shipping Phone"].loc[data_magento["Order #"][i]==data_magentoUser["Sales Order Id"]].values[0]

indeks = data_magento[data_magento['Phone'].isnull()].index.to_list()
data_magento['Phone Condition'] = np.nan
data_magento['Phone Condition'][indeks] = 'X'

data_magentoUser['Sales Order Id'] = data_magentoUser['Sales Order Id'].astype(str)
data_magento['Order #'] = data_magento['Order #'].astype(str)

temp2 = data_magento.merge(data_magentoUser[['Sales Order Id', 'AWB', 'Shipping Cost', 'Shipping Address1', 'Shipping City', 'Shipping Province', 'Shipping Phone', 'Shipping Courier']].drop_duplicates('Sales Order Id'), how = 'left', left_on = 'Order #', right_on = 'Sales Order Id').set_index(data_magento.index)

data_magento['Shipping'] = np.nan
data_magento['AWB'] = np.nan
data_magento['Shipping Courier'] = np.nan

data_magento['Shipping'][temp2.index] = temp2['Shipping Cost']
data_magento['AWB'][temp2.index] = temp2['AWB']
data_magento['Shipping Courier'][temp2.index] = temp2['Shipping Courier']


# Phone formatting
data_magento["Phone"] = data_magento["Phone"].fillna(0.0)
data_magento["Phone"] = data_magento["Phone"].astype(str).str.replace(" ","")
data_magento["Phone"] = data_magento["Phone"].astype(str).str.replace("-","")
temp = data_magento["Phone"].astype(str).str.split(",", expand = True)
data_magento["Phone"] = temp[0]
temp = data_magento["Phone"].astype(str).str.split("/", expand = True)
data_magento["Phone"] = temp[0]
data_magento["Phone"] = data_magento["Phone"].astype(str).str.replace("^\+620", "0", regex = True)
data_magento["Phone"] = data_magento["Phone"].astype(str).str.replace("^\+62", "0")
data_magento["Phone"] = data_magento["Phone"].astype(str).str.replace("^620", "0", regex = True)
data_magento["Phone"] = data_magento["Phone"].astype(str).str.replace("^62", "0", regex = True)
data_magento["Phone"] = data_magento["Phone"].astype(str).str.replace("^8", "08", regex = True)
data_magento["Phone"] = data_magento["Phone"].astype(str).str.replace("^62", "0", regex = True)
data_magento["Phone"] = data_magento["Phone"].astype(str).str.replace("(021)", "021")
data_magento["Phone"] = data_magento["Phone"].astype(str).str.replace("^21", "021", regex = True)


data_magento['Shipping Address1'] = np.nan
data_magento['Shipping City'] = np.nan
data_magento['Shipping Province'] = np.nan
data_magento['Shipping Phone'] = np.nan


data_magento["Shipping Phone"] = data_magento["Shipping Phone"].fillna(0.0)
data_magento["Shipping Phone"] = data_magento["Shipping Phone"].astype(str).str.replace(" ","")
data_magento["Shipping Phone"] = data_magento["Shipping Phone"].astype(str).str.replace("-","")
temp = data_magento["Shipping Phone"].astype(str).str.split(",", expand = True)
data_magento["Shipping Phone"] = temp[0]
temp = data_magento["Shipping Phone"].astype(str).str.split("/", expand = True)
data_magento["Shipping Phone"] = temp[0]
data_magento["Shipping Phone"] = data_magento["Shipping Phone"].astype(str).str.replace("^\+620", "0", regex = True)
data_magento["Shipping Phone"] = data_magento["Shipping Phone"].astype(str).str.replace("^\+62", "0")
data_magento["Shipping Phone"] = data_magento["Shipping Phone"].astype(str).str.replace("^620", "0", regex = True)
data_magento["Shipping Phone"] = data_magento["Shipping Phone"].astype(str).str.replace("^62", "0", regex = True)
data_magento["Shipping Phone"] = data_magento["Shipping Phone"].astype(str).str.replace("^8", "08", regex = True)
data_magento["Shipping Phone"] = data_magento["Shipping Phone"].astype(str).str.replace("^62", "0", regex = True)
data_magento["Shipping Phone"] = data_magento["Shipping Phone"].astype(str).str.replace("(021)", "021")
data_magento["Shipping Phone"] = data_magento["Shipping Phone"].astype(str).str.replace("^21", "021", regex = True)

# Master tatanama
print("--- %s seconds ---" % (time.time() - start_time))
print("Fulfilling SKU ====== 3/10")

data_magento['SKU'] = data_magento['SKU'].astype(str).str.split('-').str[0]
indeks = data_magento[~data_magento['SKU'].astype(str).isin(data_SKU['SKU'].astype(str))]['SKU'].index.to_list()

skuhd = data_magento[data_magento['SKU'].astype(str).str.contains('hd', case = False)]
skushopee = data_magento[data_magento['SKU'].astype(str).str.contains('(S)',regex = False)]
data_magento = data_magento[~data_magento['SKU'].astype(str).isin(skushopee['SKU'].astype(str))]
data_magento = data_magento[~data_magento['SKU'].astype(str).isin(skuhd['SKU'].astype(str))]

skuhd = skuhd.reset_index(drop = True)
skushopee = skushopee.reset_index(drop = True)
data_magento = data_magento.reset_index(drop = True)

indeks = data_magento[~data_magento['SKU'].astype(str).isin(data_SKU['SKU'].astype(str))]['SKU'].index.to_list()
for i in indeks:
    if data_magento['Product Name'][i].lower() in data_SKU['Nama Produk'].str.lower().values:
        data_magento['SKU'][i] = data_SKU['SKU'].loc[data_SKU['Nama Produk'].str.lower() == str(data_magento['Product Name'][i]).lower()].values[0]

list_alias = []
list_alias_name = []
for colname in data_SKU.columns:
    if 'Alias SKU' in colname:
        list_alias.append(colname)
    if 'Alias Nama' in colname:
        list_alias_name.append(colname)

for i in indeks:
    for j in list_alias:
        if str(data_magento['SKU'][i]) in data_SKU[j].astype(str).values:
            idx = data_SKU[str(data_magento['SKU'][i]) == data_SKU[j].astype(str)].index.to_list()
            for k in idx:
                if str(data_magento['SKU'][i]) == data_SKU[j.replace('SKU', 'Nama')][k]:
                    data_magento['SKU'][i] = data_SKU['SKU'][k]

indeks = data_magento[~data_magento['SKU'].astype(str).isin(data_SKU['SKU'].astype(str))]['SKU'].index.to_list()

for i in indeks:
    for j in list_alias_name:
        if str(data_magento['Product Name'][i]).lower() in data_SKU[j].astype(str).str.lower().values:
            data_magento['SKU'][i] = data_SKU['SKU'].loc[str(data_magento['Product Name'][i]).lower() == data_SKU[j].astype(str).str.lower()].values[0]

data_magento = data_magento[data_magento['Product Name'].astype(str) != 'WRP Everyday Low Fat Milk Coklat 4 Pillow Bag']
data_magento = data_magento[data_magento['Product Name'].astype(str) != 'Heavenly Kitchen Vietnamese Coffee Latte (12 Sch)']
data_magento = data_magento[data_magento['Product Name'].astype(str) != 'Heavenly Blush Yo Raspberry Pumpkin (24 pcs)']
data_magento = data_magento[data_magento['Product Name'].astype(str) != 'Heavenly Blush Yo Raspberry Pumpkin (1 pc)']
data_magento = data_magento[data_magento['Product Name'].astype(str) != 'Heavenly Kitchen Thai Tea Latte (12 Sch)']
data_magento = data_magento[data_magento['Product Name'].astype(str) != 'Heavenly Blush Greek Yogurt Blueberry (24 Pcs)']
data_magento = data_magento[data_magento['Product Name'].astype(str) != 'Heavenly Blush Greek Yogurt Blueberry']
data_magento = data_magento[data_magento['Product Name'].astype(str) != 'Heavenly Blush Yo Lychee Spinach (24 Pcs)']
data_magento = data_magento[data_magento['Product Name'].astype(str) != 'Heavenly Blush Yo Lychee Spinach (1 pc)']



data_magento['SKU'] = data_magento['SKU'].astype(str).str.replace('7300281P24', '7300281')

data_magento = data_magento[~data_magento['Product Name'].astype(str).str.contains('JANGAN DIORDER INI TESTING')]
data_magento = data_magento[~data_magento['SKU'].astype(str).str.contains('7300851')]
data_magento = data_magento[~data_magento['SKU'].astype(str).str.contains('F0301002006')]
data_magento = data_magento[~data_magento['SKU'].astype(str).str.contains('F0302004002')]
data_magento = data_magento[~data_magento['SKU'].astype(str).str.contains('F0301002005P24')]
data_magento = data_magento[~data_magento['SKU'].astype(str).str.contains('F0301002005')]


print("--- %s seconds ---" % (time.time() - start_time))
print("Listing SKU Missing ====== 4/10")   
idx = data_magento[['SKU']][data_magento['SKU'].isnull()].drop_duplicates().index.to_list()
idx = idx + data_magento[['SKU', 'Product Name']][~data_magento['SKU'].astype(str).isin(data_SKU['SKU'].astype(str))].index.to_list()
idx = list(dict.fromkeys(idx))
idx_s = skushopee[~skushopee['SKU'].astype(str).str.replace('(S)','', regex = False).isin(data_SKU['SKU'].astype(str))].index.to_list()
idx_s = list(dict.fromkeys(idx_s))
idx_hd = skuhd[~skuhd['SKU'].astype(str).str.replace('hd','',case = False).isin(data_SKU['SKU'].astype(str))].index.to_list()
idx_hd = list(dict.fromkeys(idx_hd))

to_excel = data_magento.to_excel(r'magento_new.xlsx', index = False)
print("--- %s seconds ---" % (time.time() - start_time))
if len(idx) != 0 or len(idx_s) != 0 or len(idx_hd) != 0:
    print('SKU is missing')
    alert = data_magento.iloc[idx, ][['SKU', 'Product Name']].drop_duplicates()
    alert = alert._append(skushopee.iloc[idx_s][['SKU', 'Product Name']].drop_duplicates(), ignore_index = True, sort = False)
    alert['SKU Valid'] = np.nan
    to_excel = alert.to_excel('ALERT_MAGENTO_SKU_MISSING.xlsx')
    print("Some SKU Missing Please Complete It ====== 5/10")
    print("--- %s seconds ---" % (time.time() - start_time))
    s = smtplib.SMTP('smtp.gmail.com', 587) 

    # start TLS for security 
    s.starttls() 

    # Authentication 
    s.login("automationnfi@gmail.com", "nutrifood2020") 

    msg = MIMEMultipart()
    msg['Subject'] = "ALERT SKU MAGENTO MISSING"


    html = """\
    <html>
    <head></head>
    <body>
        {0}
    </body>
    </html>
    """.format(alert.to_html())

    part1 = MIMEText(html, 'html')
    msg.attach(part1)

    # sending the mail 
    s.sendmail("automationnfi@gmail.com", "andra.miftah@nutrifood.co.id", msg.as_string()) 

    # terminating the session 
    s.quit()
else :
    print("Preparing Appending Magento to Masterdata")
    print("Filling Brand ====== 5/10")  
    data_magento = data_magento._append(skushopee, ignore_index = True, sort = False)
    data_magento = data_magento.reset_index(drop = True)

    data_magento['SKU'] = data_magento['SKU'].astype(str)
    data_magento['Product Name'] = data_magento['Product Name'].astype(str)
    data_SKU['Real SKU'] = data_SKU['SKU'].astype(str)
    data_SKU['Real Nama Produk'] = data_SKU['Nama Produk'].astype(str)

    data_magento = data_magento.merge(data_SKU[['Real SKU', 'Real Nama Produk']].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU')

    temp = data_magento[data_magento['Real SKU'].isnull()].copy()
    temp['SKU'] = temp['SKU'].astype(str).str.replace('(S)','', regex = False)
    temp = temp.merge(data_SKU[['Real SKU', 'Real Nama Produk']].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU').set_index(temp.index)
    temp['Real SKU_x'] = temp['Real SKU_x'].fillna(temp['Real SKU_y'])
    temp['Real Nama Produk_x'] = temp['Real Nama Produk_x'].fillna(temp['Real Nama Produk_y'])
    temp = temp.drop(['Real SKU_y', 'Real Nama Produk_y'], axis = 1)
    temp = temp.rename(columns = {'Real SKU_x' : 'Real SKU', 'Real Nama Produk_x' : 'Real Nama Produk'})

    indeks = data_magento[data_magento['Real SKU'].isnull()].index.to_list()
    data_magento['Real SKU'][indeks] = temp['Real SKU'][indeks]
    data_magento['Real Nama Produk'][indeks] = temp['Real Nama Produk'][indeks]

    temp = data_magento[data_magento['Real SKU'].isnull()].copy()
    temp['SKU'] = temp['SKU'].astype(str).str.replace('hd','', regex = False)
    temp = temp.merge(data_SKU[['Real SKU', 'Real Nama Produk']].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU').set_index(temp.index)
    temp['Real SKU_x'] = temp['Real SKU_x'].fillna(temp['Real SKU_y'])
    temp['Real Nama Produk_x'] = temp['Real Nama Produk_x'].fillna(temp['Real Nama Produk_y'])
    temp = temp.drop(['Real SKU_y', 'Real Nama Produk_y'], axis = 1)
    temp = temp.rename(columns = {'Real SKU_x' : 'Real SKU', 'Real Nama Produk_x' : 'Real Nama Produk'})

    indeks = data_magento[data_magento['Real SKU'].isnull()].index.to_list()
    data_magento['Real SKU'][indeks] = temp['Real SKU'][indeks]
    data_magento['Real Nama Produk'][indeks] = temp['Real Nama Produk'][indeks]

    data_magento['Real SKU'] = data_magento['Real SKU'].astype(str)
    data_magento = data_magento.merge(data_SKU[['SKU', 'Brand', 'Sub Brand', 'Parent Item', 'Parent SKU']].drop_duplicates(['SKU']), how = 'left', left_on = 'Real SKU', right_on = 'SKU')
    data_magento = data_magento.drop(['SKU_y'], axis = 1)
    data_magento = data_magento.rename(columns = {'SKU_x':'SKU'})

    print("--- %s seconds ---" % (time.time() - start_time))
    print("Unbundling ====== 6/10")
    list_col = ['SKU'] + data_SKU.columns[data_SKU.columns.get_loc('Produk 1'):data_SKU.columns.get_loc('Harga Organik 7')+1].to_list()
    data_magento = data_magento.merge(data_SKU[list_col].drop_duplicates(['SKU']), how = 'left', left_on = 'Real SKU', right_on = 'SKU')
    data_magento = data_magento.drop(['SKU_y'], axis = 1)
    data_magento = data_magento.rename(columns = {'SKU_x':'SKU'})

    indeks = data_magento[data_magento['Brand'] == 'Bundle'].index.to_list()
    data_magento['Bundle Flag'] = np.nan
    data_magento['Bundle Flag'][indeks] = 'Bundle'

    print("--- %s seconds ---" % (time.time() - start_time))
    print("Filling Date ====== 7/10")
    data_magento['Date'] = np.nan
    data_magento['Month'] = np.nan
    data_magento['Year'] = np.nan

    for i in range(data_magento.shape[0]):
        if int(data_magento['Order date'][i].strftime('%d')) <= 12:
            data_magento['Date'][i] = pd.to_datetime(data_magento['Order date'][i].strftime('%Y-%d-%m %H:%M')).day
            data_magento['Month'][i] = pd.to_datetime(data_magento['Order date'][i].strftime('%Y-%d-%m %H:%M')).month_name()
            data_magento['Year'][i] = pd.to_datetime(data_magento['Order date'][i].strftime('%Y-%d-%m %H:%M')).year
        else :
            data_magento['Date'][i] = pd.to_datetime(data_magento['Order date'][i]).day
            data_magento['Month'][i] = pd.to_datetime(data_magento['Order date'][i]).month_name()
            data_magento['Year'][i] = pd.to_datetime(data_magento['Order date'][i]).year


    quarter = pd.DataFrame([['January', 1], ['February', 1], ['March', 1], ['April', 2], ['May', 2], ['June', 2], 
            ['July', 3], ['August', 3], ['September', 3],['October', 4], ['November', 4], ['December', 4]], columns = ['Bulan', 'Quarter'])
    data_magento = data_magento.merge(quarter, how = 'left', left_on = 'Month', right_on = 'Bulan')
    data_magento = data_magento.drop(['Bulan'], axis = 1)
    data_bulan = pd.DataFrame([{'Bulan' : 'December', 'Number' : 12} ,
            {'Bulan' : 'January' , 'Number': 1},
            {'Bulan' : 'February' , 'Number': 2},
            {'Bulan' : 'March' , 'Number': 3},
            {'Bulan' : 'April' , 'Number': 4},
            {'Bulan' : 'May' , 'Number': 5},
            {'Bulan' : 'June', 'Number': 6},
            {'Bulan' : 'July' , 'Number': 7},
            {'Bulan' : 'August', 'Number' : 8},
            {'Bulan' : 'September', 'Number' : 9},
            {'Bulan' : 'October' , 'Number': 10},
            {'Bulan' : 'November' , 'Number': 11}])
    temp = data_magento.copy()
    temp['Day'] = temp['Date']
    temp = temp.merge(data_bulan, how = 'left', left_on = 'Month', right_on='Bulan')
    temp= temp.rename(columns = {'Month' : 'Bulan', 'Number' : 'Month'})
    data_magento['Week'] = pd.to_datetime(temp[['Year', 'Month', 'Day']]).dt.strftime("%U")
    data_magento['True datetime'] = pd.to_datetime(temp[['Year', 'Month', 'Day']])
    data_magento['Channel'] = 'Nutrimart'
    data_magento['Store'] = 'Nutrimart'
    data_magento['Selling Price'] = data_magento['Item Price']
    print("--- %s seconds ---" % (time.time() - start_time))
    print("Pricing")
    data_magento = data_magento.merge(data_SKU[['SKU', 'Price List NFI', 'Harga Cost']].drop_duplicates('SKU'), how = 'left', left_on = 'Real SKU', right_on = 'SKU').set_index(data_magento.index)
    data_magento = data_magento.rename(columns = {'SKU_x' : 'SKU', 'Price List NFI_x' : 'Price List NFI'})
    data_magento['Price List NFI'] = pd.to_numeric(data_magento['Price List NFI']).astype(int)
    data_magento['Harga Cost'] = pd.to_numeric(data_magento['Harga Cost'], errors = 'coerce').fillna(0).astype(int)
    data_magento['Qty. Invoiced'] = pd.to_numeric(data_magento['Qty. Invoiced']).astype(int)
    data_magento['Total Net'] = data_magento['Price List NFI'] * data_magento['Qty. Invoiced']
    data_magento['Total Harga Cost'] = data_magento['Harga Cost'] * data_magento['Qty. Invoiced']

    data_magento = data_magento.drop('SKU_y', axis = 1)
    data_magento = data_magento.reset_index(drop = True)
    data_magento['Order #'] = data_magento['Order #'].astype(str).str.replace('.0', '', regex = False)

    list_bundle = data_magento[data_magento['Bundle Flag'] == 'Bundle'][['Order #', 'Product Name', 'Subtotal', 'Total']].groupby(['Order #', 'Product Name']).sum().reset_index()
    list_nobundle = data_magento[data_magento['Bundle Name'].notnull()]
    list_nobundle = list_nobundle.merge(list_bundle, how = 'left', left_on = ['Order #', 'Bundle Name'], right_on = ['Order #', 'Product Name']).set_index(list_nobundle.index)
    list_nobundle

    data_magento['Total'][list_nobundle.index] = list_nobundle['Total_y']
    data_magento['Subtotal'][list_nobundle.index] = list_nobundle['Subtotal_y']

    temp = data_magento[data_magento['Bundle Name'].notnull()]
    temp['Subtotal'] = temp['Subtotal'] * temp['Total Harga Cost']/temp.groupby(['Order #', 'Bundle Name'])['Total Harga Cost'].transform('sum')
    temp['Total'] = temp['Total'] * temp['Total Harga Cost']/temp.groupby(['Order #', 'Bundle Name'])['Total Harga Cost'].transform('sum')
    temp['Selling Price'] = temp['Subtotal'] / temp['Qty. Invoiced']

    data_magento['Total'][temp.index] = temp['Total'].fillna(0).astype(int)
    data_magento['Subtotal'][temp.index] = temp['Subtotal'].fillna(0).astype(int)
    data_magento['Selling Price'][temp.index] = temp['Selling Price'].fillna(0).astype(int)

    print("Filling Location")
    data_magento['Kecamatan'] = np.nan
    data_magento['Kelurahan'] = np.nan

    indeks = data_magento[data_magento['City'].astype(str).str.contains('/')]['City'].index.to_list()
    if len(indeks)>0:
        data_magento['Kecamatan'][indeks] = data_magento['City'][indeks].str.split('/', n = 1,expand = True)[1]
        data_magento['City'][indeks] = data_magento['City'][indeks].str.split('/', n = 1,expand = True)[0]

    indeks = data_magento[data_magento['Kecamatan'].astype(str).str.contains('-')]['Kecamatan'].index.to_list()
    if len(indeks)>0:
        data_magento['Kelurahan'][indeks] = data_magento['Kecamatan'][indeks].str.split('-', n = 1,expand = True)[1]
        data_magento['Kecamatan'][indeks] = data_magento['Kecamatan'][indeks].str.split('-', n = 1,expand = True)[0]

    indeks = data_magento[data_magento['City'].astype(str).str.contains(',')]['City'].index.to_list()
    if len(indeks)>0:
        data_magento['Kecamatan'][indeks] = data_magento['City'][indeks].str.split(',', n = 1,expand = True)[1]
        data_magento['City'][indeks] = data_magento['City'][indeks].str.split(',', n = 1,expand = True)[0]

    indeks = data_magento[data_magento['Kecamatan'].astype(str).str.contains(',')]['Kecamatan'].index.to_list()
    if len(indeks)>0:
        data_magento['Kelurahan'][indeks] = data_magento['Kecamatan'][indeks].str.split(',', n = 1,expand = True)[1]
        data_magento['Kecamatan'][indeks] = data_magento['Kecamatan'][indeks].str.split(',', n = 1,expand = True)[0]

    city = pd.read_excel(r'All Data/list_city.xlsx')
    temp = data_magento.copy()
    temp['City'] = temp['City'].astype(str).str.lower()
    city['All City'] = city['All City'].astype(str).str.lower()
    temp = temp.merge(city.drop_duplicates('All City'), how = 'left', left_on = 'City', right_on = 'All City').set_index(temp.index)
    indeks = temp[temp['Real City'].notnull()].index.to_list()
    data_magento['City'][indeks] = temp['Real City'][indeks]

    province = pd.read_excel(r'All Data/list_province.xlsx')
    temp = data_magento.copy()
    temp['Region'] = temp['Region'].astype(str).str.lower()
    province['All Province'] = province['All Province'].astype(str).str.lower()
    temp = temp.merge(province.drop_duplicates('All Province'), how = 'left', left_on = 'Region', right_on = 'All Province').set_index(temp.index)
    indeks = temp[temp['Real Province'].notnull()].index.to_list()
    data_magento['Region'][indeks] = temp['Real Province'][indeks]

    temp = data_magento.copy()
    temp = temp[temp['Region'].isnull()]
    temp['Region'] = temp.merge(master_map, how = 'left', on = 'City').set_index(temp.index)['Province']
    data_magento['Region'][temp.index] = temp['Region']

    district = pd.read_excel(r'All Data/list_district.xlsx')
    temp = data_magento.copy()
    temp['Kecamatan'] = temp['Kecamatan'].astype(str).str.lower()
    district['All District'] = district['All District'].astype(str).str.lower()
    temp = temp.merge(district.drop_duplicates('All District'), how = 'left', left_on = 'Kecamatan', right_on = 'All District').set_index(temp.index)
    indeks = temp[temp['Real District'].notnull()].index.to_list()
    data_magento['Kecamatan'][indeks] = temp['Real District'][indeks]

    temp = data_magento.copy()
    temp2 = temp[['Region', 'City', 'Kecamatan']].merge(master_map, how = 'left', on = 'City')
    indeks = temp2[temp2['Region'] != temp2['Province']][temp2[temp2['Region'] != temp2['Province']]['City'].notnull()].index.to_list()
    data_magento['City'][indeks] = np.nan

    indeks = data_magento[data_magento['Shipping City'].astype(str).str.contains('/')]['Shipping City'].index.to_list()
    if len(indeks)>0:
        data_magento['Shipping City'][indeks] = data_magento['Shipping City'][indeks].str.split('/', n = 1,expand = True)[0]

    indeks = data_magento[data_magento['Shipping City'].astype(str).str.contains(',')]['Shipping City'].index.to_list()
    if len(indeks)>0:
        data_magento['Shipping City'][indeks] = data_magento['Shipping City'][indeks].str.split(',', n = 1,expand = True)[0]

    city = pd.read_excel(r'All Data/list_city.xlsx')
    temp = data_magento.copy()
    temp['Shipping City'] = temp['Shipping City'].astype(str).str.lower()
    city['All City'] = city['All City'].astype(str).str.lower()
    temp = temp.merge(city.drop_duplicates('All City'), how = 'left', left_on = 'Shipping City', right_on = 'All City').set_index(temp.index)
    indeks = temp[temp['Real City'].notnull()].index.to_list()
    data_magento['Shipping City'][indeks] = temp['Real City'][indeks]

    province = pd.read_excel(r'All Data/list_province.xlsx')
    temp = data_magento.copy()
    temp['Shipping Province'] = temp['Shipping Province'].astype(str).str.lower()
    province['All Province'] = province['All Province'].astype(str).str.lower()
    temp = temp.merge(province.drop_duplicates('All Province'), how = 'left', left_on = 'Shipping Province', right_on = 'All Province').set_index(temp.index)
    indeks = temp[temp['Real Province'].notnull()].index.to_list()
    data_magento['Shipping Province'][indeks] = temp['Real Province'][indeks]

    temp = data_magento.copy()
    temp = temp[temp['Shipping Province'].isnull()]
    temp['Shipping Province'] = temp.merge(master_map, how = 'left', left_on = 'Shipping City', right_on = 'City').set_index(temp.index)['Shipping Province']
    data_magento['Shipping Province'][temp.index] = temp['Shipping Province']

    temp = data_magento.copy()
    temp2 = temp[['Shipping Province', 'Shipping City']].merge(master_map, how = 'left', left_on = 'Shipping City', right_on = 'City')
    indeks = temp2[temp2['Shipping Province'] != temp2['Province']][temp2[temp2['Shipping Province'] != temp2['Province']]['Shipping City'].notnull()].index.to_list()
    data_magento['Shipping City'][indeks] = np.nan
    data_magento['Warehouse Name'] = 'Primary Warehouse'

    import os
    import glob

    list_of_files = glob.glob(r'C:\Masterdata\1. Clean Data\*.csv')
    latest_file = max(list_of_files, key=os.path.getctime)

    data_all = pd.read_csv(latest_file, sep = ';',converters = {'Phone' : str, 'Order #' : str, 'AWB' : str})
    
    #MANUAL UPLOAD DATA ALL KALAU ADA ERROR
    #data_all = pd.read_csv(r"D:\Masterdata\Clean Data\data_all_3 November With Order Online.csv", sep = ';',converters = {'Phone' : str, 'Order #' : str, 'AWB' : str})
    
    #CUTOFF DATA ALL
    #data_all = data_all[(data_all['True datetime'] >= "2024-01-01")]
    
    data_all = data_all[data_all['Store Type'] != 'Retail Online']

    data_all['Phone'] = data_all['Phone'].astype(str).str.replace('="', '', regex = False)
    data_all['Phone'] = data_all['Phone'].astype(str).str.replace('"', '', regex = False)

    lmen_store = forstok_all[forstok_all['Channel'] == 'L-Men Store Blibli']

    lmen_store = lmen_store[~lmen_store['Order #'].astype(str).isin(data_all['Order #'].astype(str))]
    lmen_store['Sales Order ID'] = lmen_store['Sales Order ID'].fillna(lmen_store['No. Order Item L-Men Blibli'])
    ornum = lmen_store[['Order #', 'Sales Order ID']].drop_duplicates('Order #')

    options = Options()
    options.add_experimental_option("prefs", {
            "download.default_directory": str(os.getcwd())  + '/Input Data',
            "download.directory_upgrade": True,
            "safebrowsing_for_trusted_sources_enabled": False,
            "safebrowsing.enabled": False
    })

    driver = webdriver.Chrome(options=options)
    driver.maximize_window()
    driver.get("https://seller.blibli.com/sign-in")
    time.sleep(30)


    username = driver.find_element(By.ID, "email")
    username.clear()
    username.send_keys("customer1@nutrimart.co.id")

    password = driver.find_element(By.ID, "password")
    password.send_keys("NutrimartNutrif00d")

    driver.find_element(By.ID, "sign-in").click()
    time.sleep(25)

    driver.get("https://seller.blibli.com/MTA/order/summary")
    WebDriverWait(driver, 40).until(EC.presence_of_element_located((By.CLASS_NAME, 'btn-primary-mta'))).click()
    driver.find_element(By.XPATH, '//*[@id="semuaFilter"]/span[1]').click()

    ornum['Phone'] = np.nan
    ornum['Email'] = np.nan
    ornum['Address'] = np.nan

    for i in ornum.index:
        orderNo = ornum['Order #'][i]
        orderNo = orderNo.replace('"','')
        orderNo = orderNo.replace('=','')
        orderItemNo = ornum['Sales Order ID'][i]
        orderItemNo = orderItemNo.replace('"','')
        orderItemNo = orderItemNo.replace('=','')
        link = "https://seller.blibli.com/MTA/neo/order/order-detail/" + "?orderNo=" + str(orderNo) + "&orderItemNo=" + str(orderItemNo)
        driver.get(link)
        WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.XPATH, '//*[@id="customer-phone"]/div/span')))
        hp = driver.find_elements(By.XPATH, '//*[@id="customer-phone"]/div/span')[0].text
        if len(driver.find_elements(By.XPATH, '//*[@id="customer-email"]/span')) != 0:
            email = driver.find_elements(By.XPATH, '//*[@id="customer-email"]/span')[0].text
            ornum['Email'][i] = email
        address = driver.find_elements(By.XPATH, '//*[@id="alamat-pengiriman"]//div//p')[0].text
        ornum['Phone'][i] = hp
        ornum['Address'][i] = address

    driver.quit()
    if len(ornum) != 0:
        tes = ornum.copy()
        tes['Real Address'] = tes['Address'].str.split('\n', expand = True)[1].str.strip()
        tes['Kelurahan'] = tes['Address'].str.split('\n', expand = True)[3].str.split('-', expand = True)[0].str.strip()
        tes['Kecamatan'] = tes['Address'].str.split('\n', expand = True)[3].str.split('-', expand = True)[1].str.strip()
        tes['City'] = tes['Address'].str.split('\n', expand = True)[4].str.split(',', expand = True)[0].str.strip()
        tes['Region'] = tes['Address'].str.split('\n', expand = True)[4].str.split(',', expand = True)[1].str.strip()
        tes['Zip Code'] = tes['Address'].str.split('\n', expand = True)[5].str.strip()

        lmen_store['Order #'] = lmen_store['Order #'].astype(str)
        tes['Order #'] = tes['Order #'].astype(str)
        lmen_store = lmen_store.merge(tes[['Order #', 'Phone', 'Email', 'Real Address', 'Kelurahan', 'Kecamatan', 'City', 'Region', 'Zip Code']].drop_duplicates('Order #'), how = 'left', on = 'Order #')

        indeks = lmen_store[lmen_store['Phone_y'].notnull()].index.to_list()
        lmen_store['Phone_x'][indeks] = lmen_store['Phone_y'][indeks]
        lmen_store['Customer Email'][indeks] = lmen_store['Email'][indeks]
        lmen_store['Phone_x'][indeks] = lmen_store['Phone_y'][indeks]
        lmen_store['Region_x'][indeks] = lmen_store['Region_y'][indeks]
        lmen_store['City_x'][indeks] = lmen_store['City_y'][indeks]
        lmen_store['Kecamatan_x'][indeks] = lmen_store['Kecamatan_y'][indeks]
        lmen_store['Kelurahan_x'][indeks] = lmen_store['Kelurahan_y'][indeks]
        lmen_store['Zip Code_x'][indeks] = lmen_store['Zip Code_y'][indeks]
        lmen_store['Address'][indeks] = lmen_store['Real Address'][indeks]

        lmen_store = lmen_store.drop(['Phone_y', 'Email', 'Region_y', 'City_y', 'Kecamatan_y', 'Kelurahan_y', 'Real Address', 'Zip Code_y'], axis = 1)
        lmen_store = lmen_store.rename(columns = {'Phone_x' : 'Phone', 'Region_x' : 'Region', 'City_x' : 'City', 'Kecamatan_x' : 'Kecamatan', 'Kelurahan_x' : 'Kelurahan', 'Zip Code_x' : 'Zip Code'})

    forstok_all = forstok_all[forstok_all['Channel'] != 'L-Men Store Blibli']
    forstok_all = forstok_all._append(lmen_store, ignore_index = True, sort = False)

    indeks = data_all[data_all['Channel'] == 'L-Men Store Blibli'].index.to_list()
    data_all['Store'][indeks] = 'Blibli'

    print('Blibli Done')
    
    #UNBUNDLING (J)
    print('Unbundling Paket J')
    
    skuj = pd.read_excel(r"C:\Users\Steven.Nathanael\Documents\2. Master Data Git\data_supp\Rumus Realisasi.xlsx")
    skuj = skuj[['SKU','Nama Paket','SKU.1','Konversi ke UOM']]
    skuj.rename(columns = {'SKU' : 'Bundle SKU',
                           'Nama Paket' : 'Bundle Name',
                           'SKU.1' : 'Real SKU',
                           'Konversi ke UOM' : 'Qty Bundle'}, inplace = True)
    skuj['Real SKU'] = skuj['Real SKU'].astype(str)

    skuj2 = pd.merge(skuj,data_SKU[['Real SKU','Brand','Real Nama Produk','Price List NFI','Sub Brand', 'Parent Item', 'Parent SKU']], on = 'Real SKU', how = 'left')
    skuj2.rename(columns = {'Real SKU' : 'Real SKU.1',
                            'Brand' : 'Brand.1',
                            'Real Nama Produk' : 'Real Nama Produk.1',
                            'Price List NFI' : 'Price List NFI.1',
                            'Sub Brand' : 'Sub Brand.1',
                            'Parent Item' : 'Parent Item.1',
                            'Parent SKU' : 'Parent SKU.1'}, inplace = True)

    skuj1 = skuj['Bundle SKU'].unique()
    skuj3 = skuj['Bundle Name'].unique()

    tes = forstok_all[forstok_all['Real SKU'].isin(skuj1)]
    tes['Bundle SKU'] = tes['Real SKU']
    tes1 = pd.merge(tes, skuj2, on = 'Bundle SKU', how = 'left')

    tes1['Real SKU'] = tes1['Real SKU.1'].astype(str)
    tes1['Real Nama Produk'] = tes1['Real Nama Produk.1']
    tes1['Qty. Invoiced.1'] = tes1['Qty. Invoiced'] * tes1['Qty Bundle']
    tes1['Qty. Invoiced'] = pd.to_numeric(tes1['Qty. Invoiced.1'])
    tes1['Brand'] = tes1['Brand.1']
    tes1['Sub Brand'] = tes1['Sub Brand.1']
    tes1['Real Nama Produk'] = tes1['Real Nama Produk.1']
    tes1['Parent Item'] = tes1['Parent Item.1']
    tes1['Parent SKU'] = tes1['Parent SKU.1'].astype(str)
    tes1['Price List NFI'] = pd.to_numeric(tes1['Price List NFI.1'].astype(int))

    tes1.rename(columns = {'Bundle Name_x' : 'Bundle Name'}, inplace = True)
    tes1['Bundle Name'] = tes1['Bundle Name_y']
    tes1.drop(columns = ['Bundle SKU','Bundle Name_y','Real SKU.1','Real Nama Produk.1','Qty. Invoiced.1','Qty Bundle','Brand.1','Sub Brand.1','Real Nama Produk.1','Parent Item.1','Parent SKU.1','Price List NFI.1'], inplace = True)

    tes1['Total Net'] = tes1['Price List NFI'] * tes1['Qty. Invoiced']
    tes1['Harga Cost'] = tes1['Price List NFI']
    tes1['Total Harga Cost'] = tes1['Total Net']

    forstok_all.loc[forstok_all['Real SKU'].str.contains('J', na = False, case = False),'Brand']= 'Bundle'
    forstok_all.loc[forstok_all['Real SKU'].str.contains('J', na = False, case = False),'Sub Brand']= np.nan
    forstok_all.loc[forstok_all['Real SKU'].str.contains('J', na = False, case = False),'Bundle Flag']= 'Bundle J'
    forstok_all = forstok_all._append(tes1, ignore_index = True)
    
    print('Done')
    
    #GABUNG MASTERDATA
    data_all = data_all[~data_all['Order #'].astype(str).isin(forstok_all['Order #'].astype(str))]
    data_all = data_all[~data_all['Order #'].astype(str).isin(data_magento['Order #'].astype(str))]
    data_all = data_all._append(forstok_all, ignore_index = True, sort = False)
    data_all = data_all._append(data_magento, ignore_index = True, sort = False)
    data_all['Brand'] = data_all['Brand'].astype(str).str.replace('Tropicana Slim', 'TS')

    indeks = data_all[data_all['Qty. Invoiced'] == 0].index.to_list()
    data_all['Qty. Invoiced'][indeks] = data_all['Qty. Shipped'][indeks]
    data_all['Total Net'][indeks] = data_all['Price List NFI'][indeks] * data_all['Qty. Invoiced'][indeks]
    indeks = data_all[data_all['Qty. Invoiced'] == 0].index.to_list()
    data_all['Qty. Invoiced'][indeks] = data_all['Qty. Ordered'][indeks]
    data_all['Total Net'][indeks] = data_all['Price List NFI'][indeks] * data_all['Qty. Invoiced'][indeks]

# #   Cleaning Phone Number
    import re 
    from re import sub            
    data_all['Phone'] = data_all['Phone'].astype(str).str.split(';', expand = True)[0]
    data_all['Phone'] = data_all['Phone'].astype(str).str.replace('.0','', regex = False)
    data_all['Phone'] = data_all['Phone'].astype(str).str.replace('+','', regex = False)
    non_phone = data_all[(data_all['Phone'].astype(str).str.startswith('62'))]['Phone'].unique()
    non_phone1 = data_all[(data_all['Phone'].astype(str).str.startswith('62'))][['Store','Phone']].drop_duplicates()
    for i in non_phone:
        index = data_all[data_all['Phone'].astype(str) == str(i)].index.to_list()
        phone = str(i).replace('62','0', 1)
        print(i, phone, sep='-')
        data_all['Phone'][index] = phone
      
    
    print('done')
    print("--- %s seconds ---" % (time.time() - start_time))
    data_all['Phone'] = data_all['Phone'].apply('="{}"'.format)
    data_all['Shipping Phone'] = data_all['Shipping Phone'].apply('="{}"'.format)
    indeks = data_all[data_all['City'] == 'nan'].index.to_list()
    data_all['City'][indeks] = np.nan

    indeks = data_all[data_all['Region'] == 'nan'].index.to_list()
    data_all['Region'][indeks] = np.nan

    indeks = data_all[data_all['Channel'] == 'FBL'].index.to_list()
    data_all['Warehouse Name'][indeks] = 'Lazada Warehouse'

    indeks = data_all[data_all['Warehouse Name'] == 'Lazada Warehouse'].index.to_list()
    data_all['Channel'][indeks] = 'FBL'

    print("Read Data Settlement")
    
    #EDIT KHUSUS PERIOD FORSTOK
    from datetime import datetime, timedelta
    start_date = datetime.today() - timedelta(days=12)
    end_date = datetime.today()
    
    print('Opening Chrome')
    driver = webdriver.Chrome()
    driver.maximize_window()
    actions = ActionChains(driver)
    driver.get("https://www.forstok.com/dashboard/users/login")
    print('Login Forstok')
    
    time.sleep(10)
    print('input email')
    username = driver.find_element(By.NAME, "email")
    username.clear()
    username.send_keys("timotius.giovandi@nutrifood.co.id")

    print('input password')
    password = driver.find_element(By.NAME, "password")
    password.send_keys("timo123")
    
    print('click')
    driver.find_element(By.XPATH, "/html/body/div[1]/div/div/div/form/input[3]").click()
    
    time.sleep(10)
    datefield = WebDriverWait(driver, 100).until(EC.element_to_be_clickable((By.XPATH,'html/body/div[1]/section/section[2]/article/div/div/aside/div/div[1]/section[4]/div/button')))
    time.sleep(20)
    driver.execute_script("return arguments[0].scrollIntoView(true);", datefield)
    time.sleep(10)
    driver.find_element(By.XPATH, "/html/body/div[1]/section/section[2]/article/div/div/aside/div/div[1]/section[4]/div/button").click()
    time.sleep(10)
    driver.find_element(By.XPATH, '//*[@id="exportSalesOrderForm"]/div/section/section/div[1]/section/article/article[1]/div/section/div/div[2]').click()
    pilih=0
    while pilih < 3:    
            n_elem=0
            for i in driver.find_elements(By.XPATH, '(//*[@class="DateRangePicker__MonthHeaderLabel DateRangePicker__MonthHeaderLabel--month"])'):
                val=(driver.find_elements(By.XPATH, '(//*[@class="DateRangePicker__MonthHeaderLabel DateRangePicker__MonthHeaderLabel--month"])')[n_elem].text.split()[0])
                if val==start_date.strftime('%B'):
                    fin_n_elem=n_elem
                n_elem=n_elem+1
            if fin_n_elem==0:
                text = '(//*[@class="DateRangePicker__Month"])[1]//*[@class="DateRangePicker__DateLabel" and (text()="' + str(start_date.day) + '")]'
                if len(driver.find_elements(By.XPATH, text))==2:
                    text = '('+text+')[2]'
            else:
                text = '(//*[@class="DateRangePicker__Month"])[2]//*[@class="DateRangePicker__DateLabel" and (text()="' + str(start_date.day) + '")]'
                if len(driver.find_elements(By.XPATH, text))==2:
                    text = '('+text+')[1]'
            driver.find_element(By.XPATH, text).click()
            time.sleep(10)
    
            n_elem=0
            for i in driver.find_elements(By.XPATH, '(//*[@class="DateRangePicker__MonthHeaderLabel DateRangePicker__MonthHeaderLabel--month"])'):
                val=(driver.find_elements(By.XPATH, '(//*[@class="DateRangePicker__MonthHeaderLabel DateRangePicker__MonthHeaderLabel--month"])')[n_elem].text.split()[0])
                if val==end_date.strftime('%B'):
                    fin_n_elem=n_elem
                n_elem=n_elem+1
            if fin_n_elem==0:
                text = '(//*[@class="DateRangePicker__Month"])[1]//*[@class="DateRangePicker__DateLabel" and (text()="' + str(end_date.day) + '")]'
                if len(driver.find_elements(By.XPATH, text))==2:
                    text = '('+text+')[2]'
            else:
                text = '(//*[@class="DateRangePicker__Month"])[2]//*[@class="DateRangePicker__DateLabel" and (text()="' + str(end_date.day) + '")]'
                if end_date.strftime('%B')==start_date.strftime('%B'):
                    text = '('+text+')'#[2]
                else :
                    text = '('+text+')[1]'
            driver.find_element(By.XPATH, text).click()
            time.sleep(10)
            pilih=pilih+1
    driver.find_element(By.XPATH, '//*[@type="button" and contains(text(),"Apply")]').click()
    time.sleep(30)
    driver.find_element(By.XPATH, '/html/body/div[1]/section/section[2]/div/form/div/section/section/div[2]/aside[2]/button[2]').click()
    time.sleep(2)
    driver.quit()
    
    print('Forstok Downloaded')

    print('Download Data SKU')
    # Import library

    import time
    import pandas as pd
    import numpy as np
    import math

    import smtplib 
    from email.mime.text import MIMEText
    from email.mime.application import MIMEApplication
    from email.mime.multipart import MIMEMultipart
    from smtplib import SMTP
    import smtplib
    import sys
    import requests
    import os

    startdate = start_date-timedelta(days=1)
    startdate1 = startdate.strftime("%Y-%m-%d")
    enddate = end_date.strftime("%Y-%m-%d")

    text = 'https://forstok-staging-storage.s3.ap-southeast-1.amazonaws.com/items_import/Nutrifood-forstok-sales_orders-version_2-' + startdate1 + 'T17:00:00Z-' + enddate + 'T16:59:59Z.xlsx'
    print(text)
    while True:
        r = s.get(text)
        print('Waiting to Download Forstok 2')

        if r.status_code == requests.codes.ok:
            print(r.status_code)
            with open('Input Data/Forstok_new_input.xls', 'wb') as output:
                output.write(r.content)
            print('output out')
            break
        else :
            time.sleep(30)

    with open('Input Data/nutrifood--forstok-sales_orders-' + str(date) + '.xls', 'wb') as output:
        output.write(r.content)
        
    orstat_forstok = pd.read_excel(r'Input Data/Forstok_new_input.xls')
    orstat_forstok['Date'] = np.nan
    orstat_forstok['Month'] = np.nan
    orstat_forstok['Year'] = np.nan

    #MANUAL UPLOAD KALAU ERROR
    #orstat_forstok = pd.read_excel(r"C:\Users\steven.nathanael\Downloads\Nutrifood-forstok-sales_orders-version_2-2023-05-14T17 00 00Z-2023-05-29T16 59 59Z.xlsx")

    orstat_forstok.rename(columns = {'Shipping Label Printed Date' : 'Printed Date',
                              'Shipping Date' : 'Fulfilled Date',
                              'SKU Code' : 'SKU',
                              'Shipping Fee (Non-cashless)' : 'Shipping',
                              'Voucher Seller' : 'Seller Voucher',
                              'Platform Rebate' : 'Channel Rebate',
                              'Order Note' : 'Note'}, inplace = True)
    orstat_forstok["Order Date"] = orstat_forstok["Order Date"].str.replace(' WIB', '')
    orstat_forstok["Payment Date"] = orstat_forstok["Payment Date"].str.replace(' WIB', '')

    orstat_forstok['Order Date'] = pd.to_datetime(orstat_forstok['Order Date'], format = '%d/%m/%Y %H:%M' )
    for i in range(orstat_forstok.shape[0]):
        if int(orstat_forstok['Order Date'][i].strftime('%d')) <= 12:
            orstat_forstok['Date'][i] = pd.to_datetime(orstat_forstok['Order Date'][i].strftime('%Y-%d-%m %H:%M')).day
            orstat_forstok['Month'][i] = pd.to_datetime(orstat_forstok['Order Date'][i].strftime('%Y-%d-%m %H:%M')).month_name()
            orstat_forstok['Year'][i] = pd.to_datetime(orstat_forstok['Order Date'][i].strftime('%Y-%d-%m %H:%M')).year
        else :
            orstat_forstok['Date'][i] = pd.to_datetime(orstat_forstok['Order Date'][i]).day
            orstat_forstok['Month'][i] = pd.to_datetime(orstat_forstok['Order Date'][i]).month_name()
            orstat_forstok['Year'][i] = pd.to_datetime(orstat_forstok['Order Date'][i]).year

    temp = orstat_forstok.copy()
    temp['Day'] = temp['Date']
    temp = temp.merge(data_bulan, how = 'left', left_on = 'Month', right_on='Bulan')
    temp= temp.rename(columns = {'Month' : 'Bulan', 'Number' : 'Month'})
    orstat_forstok['Week'] = pd.to_datetime(temp[['Year', 'Month', 'Day']]).dt.strftime("%U")
    temp['Hour'] = pd.to_datetime(orstat_forstok['Order Date']).dt.hour
    temp['Minute'] = pd.to_datetime(orstat_forstok['Order Date']).dt.minute
    temp['Second'] = pd.to_datetime(orstat_forstok['Order Date']).dt.second
    orstat_forstok['True datetime'] = pd.to_datetime(temp[['Year', 'Month', 'Day', 'Hour', 'Minute', 'Second']])

    print("Update Order Status")
    date_min  = orstat_forstok['True datetime'].min()
    orstat_forstok = orstat_forstok[['Sales Order ID','Fulfillment Status']].drop_duplicates('Sales Order ID')
    shopee_bp = data_all[data_all['Customer Name'] == 'Shopee Brand Portal']
    data_all = data_all[data_all['Customer Name'] != 'Shopee Brand Portal']
    data_all['True datetime1'] = pd.to_datetime(data_all['True datetime'], format='ISO8601')
    temp_all = data_all[data_all['True datetime1'] >= pd.to_datetime(date_min)][~data_all[data_all['True datetime1'] >= pd.to_datetime(date_min)]['Channel'].isin(['Nutrimart', 'L-Men Store Blibli', 'Order Online', 'Order Online Jakarta', 'Order Online Surabaya'])]
    orstat_forstok['Sales Order ID'] = orstat_forstok['Sales Order ID'].astype(str)
    temp_all['Sales Order ID'] = temp_all['Sales Order ID'].astype(str)
    temp_all = temp_all.merge(orstat_forstok, how = 'left', on = 'Sales Order ID')
    temp_all['Fulfillment Status'] = temp_all['Fulfillment Status'].fillna('Canceled')
    temp_all['Order Status'] = temp_all['Fulfillment Status']
    temp_all = temp_all.drop('Fulfillment Status', axis = 1)
    data_all['Order #'] = data_all['Order #'].astype(str)
    temp_all['Order #'] = temp_all['Order #'].astype(str)
    data_all = data_all[~data_all['Order #'].isin(temp_all['Order #'])]
    data_all = data_all._append(temp_all, ignore_index = True, sort = False)
    data_all = data_all._append(shopee_bp, ignore_index = True, sort = False)
    print("Export Master Data")

Starting Magento
Opening Chrome
Login Magento
Go To Sales Form
2024-01-26 18:00:34.413611
2024-02-07 18:00:34.413611
Input Date
Download Magento


In [ ]:
### if sku missing di magento
### to do : listing di tatanama rerun kode atas Run -4 
print(data_magento[~data_magento['SKU'].isin(data_SKU['SKU'])][['SKU','Product Name']].drop_duplicates())
print(data_magento[~data_magento['SKU'].isin(data_SKU['SKU'])]['Product Name'].unique())

In [68]:
data_all[['Year','Month']].drop_duplicates()

,Year,Month
0,2023.0,January
74705,2023.0,February
263512,2023.0,March
382120,2023.0,April
558988,2023.0,May
683648,2023.0,June
886093,2023.0,July
1045826,2023.0,August
1220504,2023.0,September
1449009,2023.0,October


In [78]:
lmen['True datetime']

0     2024-01-08
1     2024-01-08
2     2024-01-08
3     2024-01-08
4     2024-01-08
         ...    
291   2024-01-18
292   2024-01-22
293   2024-01-29
294   2024-02-02
295   2024-02-03
Name: True datetime, Length: 296, dtype: datetime64[ns]

In [79]:
data_all['True datetime'].unique()

array(['2023-01-02 00:00:00', '2023-01-01 00:00:00',
       '2023-01-03 00:00:00', ..., Timestamp('2024-01-28 00:03:00'),
       Timestamp('2024-01-28 00:02:00'), Timestamp('2024-01-28 00:01:00')],
      dtype=object)

In [74]:
batas = pd.to_datetime('2023-03-01')

In [66]:
## run-5
indeks = data_all[data_all['Qty. Invoiced'].isnull()].index.to_list()
data_all['Qty. Invoiced'][indeks] = 0
data_all['Total Net'][indeks] = data_all['Price List NFI'][indeks] * data_all['Qty. Invoiced'][indeks]

In [ ]:
#TES NEW PKP
tes = pd.read_csv(r"C:\Users\steven.nathanael\Downloads\Main Sheet (timo) (2)_Full Data_data (20).csv")
tes[(tes['Year of Invoice Date SO'].isin([2023, 2024]))&(tes['Month of Invoice Date SO'].isin(['December','January']))][['Customer SO Parent Desc','Customer SO Desc']].drop_duplicates()